In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mkdir -p /content/drive/MyDrive/colab_data


In [ ]:
from google.colab import files
uploaded = files.upload()  # pick your .parquet files


Saving part.144.parquet to part.144.parquet
Saving part.154.parquet to part.154.parquet
Saving part.222.parquet to part.222.parquet
Saving part.669.parquet to part.669.parquet
Saving part.679.parquet to part.679.parquet
Saving part.701.parquet to part.701.parquet
Saving part.711.parquet to part.711.parquet
Saving part.864.parquet to part.864.parquet
Saving part.874.parquet to part.874.parquet
Saving part.1022.parquet to part.1022.parquet
Saving part.1032.parquet to part.1032.parquet
Saving part.1285.parquet to part.1285.parquet
Saving part.1295.parquet to part.1295.parquet
Saving part.1344.parquet to part.1344.parquet
Saving part.1354.parquet to part.1354.parquet
Saving part.1469.parquet to part.1469.parquet
Saving part.1479.parquet to part.1479.parquet
Saving part.1501.parquet to part.1501.parquet
Saving part.1511.parquet to part.1511.parquet
Saving part.1667.parquet to part.1667.parquet
Saving part.1677.parquet to part.1677.parquet
Saving part.1902.parquet to part.1902.parquet
Saving

In [ ]:
import shutil, pathlib

for name in uploaded.keys():
    shutil.move(name, f"/content/drive/MyDrive/colab_data/{pathlib.Path(name).name}")

print("✅ Files moved to /content/drive/MyDrive/colab_data")

✅ Files moved to /content/drive/MyDrive/colab_data


#  Keyword Normalization Check

###  Purpose
This cell verifies whether the **keyword normalization step** was successfully applied to the cleaned Parquet files.  
It checks for any leftover patterns that should have been removed during preprocessing, such as:
- `.com` domains  
- punctuation marks (`+`, `-`, `"`, `'`)  
- standalone stopwords (`in`, `a`)

---

###  Input
- Sampled cleaned Parquet files from:  
  `/content/drive/MyDrive/colab_data/cleaned_cpu/part.*.parquet`  
  Each file contains a `keyword` column.

---

###  Transformation
- Reads a few cleaned Parquet files (`part.*.parquet`) using **Polars**.  
- Applies a regular expression search for uncleaned tokens.  
- Counts and displays any suspicious keywords still containing unwanted patterns.  

---

###  Output
- Prints a summary indicating whether the normalization worked:
  -  If no uncleaned keywords remain → normalization succeeded.  
  -  If any `.com`, `+`, `-`, quotes, or stopwords remain → normalization may need to be rerun.
- No new files are created or modified.


In [ ]:
# One self-contained cell: Polars-only (CPU), vectorized keyword normalization

# 1) Setup
import os, glob, sys, subprocess

DATA_DIR = "/content/drive/MyDrive/colab_data"
OUT_DIR  = os.path.join(DATA_DIR, "cleaned_cpu")
os.makedirs(OUT_DIR, exist_ok=True)

FILES = sorted(glob.glob(os.path.join(DATA_DIR, "part.*.parquet")))
print(f"Found {len(FILES)} parquet parts in {DATA_DIR}")

# 2) Install deps (quiet)
def _pip_install(pkgs):
    return subprocess.run([sys.executable, "-m", "pip", "install", "-q", *pkgs], check=False).returncode == 0

_pip_install(["polars>=1.0.0", "pyarrow"])

import polars as pl

# 3) Vectorized transform that mirrors your unify_keyword() exactly
def pl_unify_keyword(expr: pl.Expr) -> pl.Expr:
    s = expr.cast(pl.Utf8).str.to_lowercase()

    # Replace ".com" and punctuation . - + " ' with space
    s = s.str.replace_all(r"\.com|[.\-\+\"']", " ")

    # Remove standalone 'in' and 'a'
    s = s.str.replace_all(r"\b(in|a)\b", "")

    # Normalize spaces and trim
    s = s.str.replace_all(r"\s+", " ").str.strip_chars()

    # Presence flags
    has_car     = s.str.contains(r"\bcar\b")
    has_rental  = s.str.contains(r"\brental\b")
    has_airport = s.str.contains(r"\bairport\b")

    # Remove these tokens from the tail
    tail = s.str.replace_all(r"\b(car|rental|airport)\b", "")
    tail = tail.str.replace_all(r"\s+", " ").str.strip_chars()

    # Build prefix: 'car rental' if both present, then 'airport' if present
    prefix = pl.when(has_car & has_rental).then(pl.lit("car rental")).otherwise(pl.lit(""))
    prefix = (prefix + pl.lit(" ") + pl.when(has_airport).then(pl.lit("airport")).otherwise(pl.lit("")))
    prefix = prefix.str.replace_all(r"\s+", " ").str.strip_chars()

    # Combine prefix + tail, cleanup
    out = (prefix + pl.lit(" ") + tail).str.replace_all(r"\s+", " ").str.strip_chars()
    return out

# 4) Process all files
for fp in FILES:
    name = os.path.basename(fp)
    print(f"CPU Processing: {name}")
    df = pl.read_parquet(fp)
    if "keyword" not in df.columns:
        print("  -> Skipped (no 'keyword' column).")
        continue

    df = df.with_columns(keyword = pl_unify_keyword(pl.col("keyword")))
    out_fp = os.path.join(OUT_DIR, name)
    df.write_parquet(out_fp, compression="snappy")
    print(f"  -> Saved: {out_fp}")

print(f"Done. Cleaned files in: {OUT_DIR}")


In [ ]:
# !pip -q install polars pyarrow

from pathlib import Path
import re
import polars as pl

# -------- paths --------
BASE = Path("/content/drive/MyDrive/colab_data/cleaned_cpu")
IN_ROWS = BASE / "weekly_rows_enriched_filtered_110.parquet"
OUT_AGG = BASE / "weekly_aggregated_by_week_keyword.parquet"

if not IN_ROWS.exists():
    raise FileNotFoundError(IN_ROWS)

# -------- helper --------
def norm_suffix(s: str) -> str:
    s2 = str(s).lower().strip()
    s2 = re.sub(r"\s+", "_", s2)
    s2 = re.sub(r"[^a-z0-9_]+", "_", s2)
    s2 = re.sub(r"_+", "_", s2).strip("_")
    return s2 or "unknown"

# -------- scan main file --------
lf = pl.scan_parquet(str(IN_ROWS)).select(
    pl.col("week").cast(pl.Utf8),
    pl.col("keyword").cast(pl.Utf8),
    pl.col("devicecategory").cast(pl.Utf8, strict=False),
    pl.col("searchtype").cast(pl.Utf8, strict=False),
    pl.col("adcost").cast(pl.Float64, strict=False),
    pl.col("adclicks").cast(pl.Float64, strict=False),
    pl.col("impressions").cast(pl.Float64, strict=False),
    pl.col("avg_sim_top25_this_week").cast(pl.Float64, strict=False),
    pl.col("avg_sim_top25_last_week").cast(pl.Float64, strict=False),
    pl.col("n_sim_this_week").cast(pl.Int64, strict=False),
    pl.col("n_sim_last_week").cast(pl.Int64, strict=False),
)

# -------- aggregate base stats lazily --------
facts = (
    lf.group_by(["week", "keyword"])
      .agg(
          pl.col("adcost").sum().alias("adcost_sum"),
          pl.col("adclicks").sum().alias("adclicks_sum"),
          pl.col("impressions").sum().alias("impressions_sum"),
          pl.col("avg_sim_top25_this_week").first().alias("avg_sim_top25_this_week"),
          pl.col("avg_sim_top25_last_week").first().alias("avg_sim_top25_last_week"),
          pl.col("n_sim_this_week").first().alias("n_sim_this_week"),
          pl.col("n_sim_last_week").first().alias("n_sim_last_week"),
      )
      .with_columns(
          pl.when(pl.col("adclicks_sum") > 0)
            .then(pl.col("adcost_sum") / pl.col("adclicks_sum"))
            .otherwise(None)
            .alias("cpc_week")
      )
)

# -------- devicecategory & searchtype counts (eager, small) --------
df_dev = (
    lf.group_by(["week", "keyword", "devicecategory"])
      .agg(pl.len().alias("count"))
      .collect()
)
df_st = (
    lf.group_by(["week", "keyword", "searchtype"])
      .agg(pl.len().alias("count"))
      .collect()
)

# pivot device categories wide
if len(df_dev) > 0:
    df_dev_wide = (
        df_dev.pivot(index=["week", "keyword"], columns="devicecategory", values="count")
        .fill_null(0)
    )
    dev_cols = [c for c in df_dev_wide.columns if c not in ("week", "keyword")]
    df_dev_wide = df_dev_wide.rename({c: f"n_dev_{norm_suffix(c)}" for c in dev_cols})
else:
    df_dev_wide = pl.DataFrame({"week": [], "keyword": []})

# pivot search types wide
if len(df_st) > 0:
    df_st_wide = (
        df_st.pivot(index=["week", "keyword"], columns="searchtype", values="count")
        .fill_null(0)
    )
    st_cols = [c for c in df_st_wide.columns if c not in ("week", "keyword")]
    df_st_wide = df_st_wide.rename({c: f"n_st_{norm_suffix(c)}" for c in st_cols})
else:
    df_st_wide = pl.DataFrame({"week": [], "keyword": []})

# -------- collect lazy facts (large) --------
df_facts = facts.collect(streaming=True)

# -------- join all (eager) --------
df_agg = (
    df_facts.join(df_dev_wide, on=["week", "keyword"], how="left")
             .join(df_st_wide, on=["week", "keyword"], how="left")
             .fill_null(0)
)

# -------- reorder and save --------
base_cols = [
    "week", "keyword",
    "impressions_sum", "cpc_week",
    "avg_sim_top25_this_week", "avg_sim_top25_last_week",
    "n_sim_this_week", "n_sim_last_week",
]
extra_cols = [c for c in df_agg.columns if c not in base_cols]
df_out = df_agg.select(base_cols + sorted(extra_cols))

df_out.write_parquet(str(OUT_AGG), compression="zstd")
print(f"✅ Wrote aggregated file: {OUT_AGG}")
print(f"shape: {df_out.shape}")
print(df_out.head(10))


# Aggregation of Weekly Keyword-Level Data

### Purpose
This cell aggregates the enriched weekly dataset into a more compact format containing one row per combination of **week** and **keyword**.  
The goal is to compute summary statistics such as total ad cost, clicks, impressions, and derived metrics like the weekly cost-per-click (CPC).  
Additionally, it preserves semantic similarity metrics and creates count features for device category and search type.

---

### Input
- **Input file:**  
  `/content/drive/MyDrive/colab_data/cleaned_cpu/weekly_rows_enriched_filtered_110.parquet`  
  This file contains detailed weekly observations with columns such as `week`, `keyword`, `adcost`, `adclicks`, `impressions`, and similarity metrics.

---

### Transformations
1. **Casting and Selection:**  
   Relevant columns are selected and cast to appropriate data types (string, float, integer).  

2. **Aggregation by (week, keyword):**  
   - `adcost_sum`, `adclicks_sum`, and `impressions_sum` are calculated by summing the corresponding values.  
   - The first occurrence of semantic similarity metrics (`avg_sim_top25_*` and `n_sim_*`) is retained.  
   - A new variable `cpc_week` (weekly cost-per-click) is computed as `adcost_sum / adclicks_sum`.

3. **Category Pivoting:**  
   - `devicecategory` and `searchtype` are counted and pivoted into wide format (e.g., `n_dev_desktop`, `n_st_branded_search`).

4. **Merging and Cleaning:**  
   - All aggregates and pivoted counts are joined together on `week` and `keyword`.  
   - Missing values are filled with zeros.

5. **Reordering Columns:**  
   - Base metric columns are placed first, followed by additional device and search type features.

---

### Output
- **Output file:**  
  `/content/drive/MyDrive/colab_data/cleaned_cpu/weekly_aggregated_by_week_keyword.parquet`  
  - Contains one row per (week, keyword) pair.  
  - Includes aggregated metrics and auxiliary counts.  
  - Typical columns:  
    `week`, `keyword`, `impressions_sum`, `cpc_week`, `avg_sim_top25_this_week`, `avg_sim_top25_last_week`, `n_sim_this_week`, `n_sim_last_week`, `n_dev_mobile`, `n_st_branded_search`, etc.


In [ ]:
# !pip -q install polars pyarrow sentence-transformers tqdm

from pathlib import Path
import math, gc
import numpy as np
import polars as pl
import pandas as pd
from tqdm.auto import tqdm

import torch
from sentence_transformers import SentenceTransformer

# ------------------ CONFIG ------------------
BASE = Path("/content/drive/MyDrive/colab_data/cleaned_cpu")
IN_PARQUET = BASE / "weekly_processed.parquet"   # existing cleaned file (week, keyword, devicecategory, adcost, adclicks, impressions, searchtype)

MIN_WEEKS = 110           # <-- new requirement
TOP_FRAC  = 0.25          # top 25% most similar UNIQUE neighbors
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
BATCH_SIZE = 1024
EMB_DTYPE  = np.float16

# Output files for the ≥110-weeks build
IDX_CSV   = BASE / f"keyword_index_{MIN_WEEKS}.csv"
EMB_NPY   = BASE / f"keyword_embeddings_{MIN_WEEKS}.npy"
SIM_NPY   = BASE / f"keyword_similarity_matrix_{MIN_WEEKS}.npy"
METRICS   = BASE / f"weekly_semantic_metrics_weighted_{MIN_WEEKS}.parquet"
ENRICHED  = BASE / f"weekly_rows_enriched_{MIN_WEEKS}.parquet"
ENRICHED_FILT = BASE / f"weekly_rows_enriched_filtered_{MIN_WEEKS}.parquet"
# --------------------------------------------

# ---------- sanity ----------
if not IN_PARQUET.exists():
    raise FileNotFoundError(IN_PARQUET)

# ---------- Step 1: recompute keyword list ≥ MIN_WEEKS ----------
print(f"📥 Loading {IN_PARQUET.name} and computing keywords with ≥{MIN_WEEKS} weeks…")
lf = pl.scan_parquet(str(IN_PARQUET)).select(
    pl.col("week").cast(pl.Utf8),
    pl.col("keyword").cast(pl.Utf8)
)

kw_weeks = (
    lf.unique(subset=["keyword","week"])
      .group_by("keyword")
      .agg(pl.len().alias("n_weeks"))
      .filter(pl.col("n_weeks") >= MIN_WEEKS)
      .sort("keyword")
      .collect(streaming=True)
)

K = kw_weeks.height
print(f"✅ Keywords with ≥{MIN_WEEKS} weeks: {K:,}")
if K == 0:
    raise RuntimeError("No keywords meet the new threshold. Check MIN_WEEKS.")

keywords = kw_weeks["keyword"].to_list()
n_weeks_vec = kw_weeks["n_weeks"].to_list()
pd.DataFrame({"index": range(K), "keyword": keywords, "n_weeks": n_weeks_vec}).to_csv(IDX_CSV, index=False)
print(f"💾 Wrote index: {IDX_CSV}")

# ---------- Step 2: embeddings ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🧠 Loading model '{MODEL_NAME}' on {device} …")
model = SentenceTransformer(MODEL_NAME, device=device)
torch.backends.cudnn.benchmark = True
if hasattr(torch, "set_float32_matmul_precision"):
    torch.set_float32_matmul_precision("high")

use_amp = (device == "cuda")
amp_dtype = torch.bfloat16 if (device=="cuda" and torch.cuda.get_device_capability()[0] >= 8) else torch.float16

def embed_texts(texts, batch_size=BATCH_SIZE):
    out = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Encoding keywords"):
        batch = texts[i:i+batch_size]
        with torch.inference_mode():
            if use_amp:
                with torch.autocast(device_type="cuda", dtype=amp_dtype):
                    E = model.encode(
                        batch,
                        batch_size=len(batch),
                        convert_to_numpy=True,
                        show_progress_bar=False,
                        normalize_embeddings=True,  # L2 normalized
                    )
            else:
                E = model.encode(
                    batch,
                    batch_size=len(batch),
                    convert_to_numpy=True,
                    show_progress_bar=False,
                    normalize_embeddings=True,
                )
        out.append(E.astype(np.float32, copy=False))
    return np.vstack(out)

print("🔧 Computing embeddings …")
emb_fp32 = embed_texts(keywords, batch_size=BATCH_SIZE)  # (K, d)
d = emb_fp32.shape[1]
emb = emb_fp32.astype(EMB_DTYPE, copy=False)
np.save(EMB_NPY, emb)
del emb_fp32; gc.collect()
print(f"💾 Saved embeddings: {EMB_NPY}  shape={emb.shape}, dtype={emb.dtype}")

# ---------- Step 3: full cosine matrix (small enough at K≈1.8k) ----------
print("📐 Building full cosine similarity matrix …")
# embeddings are normalized, so cosine = dot
E32 = emb.astype(np.float32, copy=False)
S = (E32 @ E32.T).astype(EMB_DTYPE, copy=False)
np.save(SIM_NPY, S)
print(f"💾 Saved similarity matrix: {SIM_NPY}  shape={S.shape}, dtype={S.dtype}")
del E32, S; gc.collect()

# ---------- Step 4: weekly weighted metrics (top 25% by similarity of UNIQUE neighbors) ----------
print("🧮 Computing weekly weighted metrics …")

def prev_week_iso(ww_yyyy: str):
    from datetime import date, timedelta
    try:
        w, y = ww_yyyy.split("-")
        d = date.fromisocalendar(int(y), int(w), 1)
        d_prev = d - timedelta(days=7)
        iso = d_prev.isocalendar()
        return f"{iso.week:02d}-{iso.year}"
    except Exception:
        return None

def topk_by_similarity_weighted_mean(sim_vals: np.ndarray, counts: np.ndarray, frac: float):
    M = sim_vals.size
    if M == 0:
        return float("nan"), 0
    k = int(math.ceil(frac * M))
    k = max(1, min(k, M))
    order = np.argsort(-sim_vals)[:k]
    w = counts[order].astype(np.int64, copy=False)
    s = sim_vals[order].astype(np.float32, copy=False)
    total = int(w.sum())
    if total <= 0:
        return float("nan"), 0
    return float((s * w).sum() / total), total

# Read rows again and restrict to new keyword set
df_rows = pl.scan_parquet(str(IN_PARQUET)).with_columns(
    pl.col("week").cast(pl.Utf8),
    pl.col("keyword").cast(pl.Utf8)
)

df_rows = df_rows.filter(pl.col("keyword").is_in(pl.Series("kw_ok", keywords)))
rows_head = df_rows.head(1_000).collect().height  # touch pipeline
print(f"Rows referencing the ≥{MIN_WEEKS}-weeks keyword set (sample touched): {rows_head}")

# Per-week multiplicities for UNIQUE neighbors
wk_counts = (
    df_rows.select("week","keyword")
           .group_by(["week","keyword"])
           .len()
           .rename({"len":"count"})
           .collect(streaming=True)
)

weeks = wk_counts.select("week").unique().to_series().to_list()
kw2idx = {kw: i for i, kw in enumerate(keywords)}
sim = np.load(SIM_NPY, mmap_mode="r")  # (K, K), float16

# Build week -> arrays
week_idx, week_cnt, week_kws, week_pos = {}, {}, {}, {}
for w in weeks:
    sub = wk_counts.filter(pl.col("week")==w)
    kws = sub["keyword"].to_list()
    cnt = np.asarray(sub["count"].to_numpy(), dtype=np.int32)
    idxs = np.asarray([kw2idx[k] for k in kws], dtype=np.int32)
    week_kws[w] = kws
    week_cnt[w] = cnt
    week_idx[w] = idxs
    week_pos[w] = {idxs[i]: i for i in range(len(idxs))}

records = []
for w in tqdm(weeks, desc="Weeks"):
    idxs_this = week_idx[w]
    cnts_this = week_cnt[w]
    kws_this  = week_kws[w]
    n = len(idxs_this)

    if n >= 1:
        S_this = np.asarray(sim[np.ix_(idxs_this, idxs_this)], dtype=np.float32)
        if n > 0:
            np.fill_diagonal(S_this, -np.inf)
    else:
        S_this = np.empty((0,0), dtype=np.float32)

    w_prev = prev_week_iso(w)
    if w_prev in week_idx:
        idxs_prev = week_idx[w_prev]
        cnts_prev = week_cnt[w_prev]
        pos_prev  = week_pos[w_prev]
        S_last = np.asarray(sim[np.ix_(idxs_this, idxs_prev)], dtype=np.float32)
    else:
        idxs_prev = np.empty((0,), dtype=np.int32)
        cnts_prev = np.empty((0,), dtype=np.int32)
        pos_prev = {}
        S_last = np.empty((n,0), dtype=np.float32)

    for i in range(n):
        kw_i = kws_this[i]
        # THIS WEEK
        if n >= 2:
            sim_row = S_this[i]
            counts  = cnts_this.copy()
            counts[i] = 0
            mask = counts > 0
            mean_this, used_this = topk_by_similarity_weighted_mean(sim_row[mask], counts[mask], TOP_FRAC)
        else:
            mean_this, used_this = float("nan"), 0

        # LAST WEEK
        if idxs_prev.size > 0:
            sim_prev = S_last[i]
            counts_prev = cnts_prev.copy()
            idx_i = kw2idx[kw_i]
            jprev = pos_prev.get(idx_i, None)
            if jprev is not None:
                counts_prev[jprev] = 0
            mask_prev = counts_prev > 0
            mean_last, used_last = topk_by_similarity_weighted_mean(sim_prev[mask_prev], counts_prev[mask_prev], TOP_FRAC)
        else:
            mean_last, used_last = float("nan"), 0

        records.append((w, kw_i, float(mean_this), float(mean_last), int(used_this), int(used_last)))

metrics_df = pl.DataFrame(
    records,
    schema=[
        ("week", pl.Utf8),
        ("keyword", pl.Utf8),
        ("avg_sim_top25_this_week", pl.Float32),
        ("avg_sim_top25_last_week", pl.Float32),
        ("n_sim_this_week", pl.Int32),
        ("n_sim_last_week", pl.Int32),
    ]
)
metrics_df.write_parquet(METRICS, compression="zstd", statistics=True)
print(f"💾 Wrote metrics: {METRICS}")

# ---------- Step 5: attach metrics to every original row (no aggregation) ----------
print("🔗 Attaching metrics to each original row (no aggregation)…")
lf_rows_full = pl.scan_parquet(str(IN_PARQUET)).with_columns(
    pl.col("week").cast(pl.Utf8),
    pl.col("keyword").cast(pl.Utf8)
)
lf_metr = pl.scan_parquet(str(METRICS))

enriched_rows = lf_rows_full.join(lf_metr, on=["week","keyword"], how="left")
enriched_rows.sink_parquet(str(ENRICHED), compression="zstd", statistics=True, row_group_size=8192)
print(f"💾 Wrote enriched rows: {ENRICHED}")

# ---------- Step 6: filter to the ≥MIN_WEEKS keyword set ----------
print("🧹 Filtering enriched rows to keywords in the new index…")
kw_set = set(keywords)
lf_enr = pl.scan_parquet(str(ENRICHED))
before_rows = lf_enr.select(pl.len()).collect().item()
before_uks  = lf_enr.select(pl.col("keyword").n_unique()).collect().item()
lf_enr_f = lf_enr.filter(pl.col("keyword").is_in(list(kw_set)))
lf_enr_f.sink_parquet(str(ENRICHED_FILT), compression="zstd", statistics=True, row_group_size=8192)
after_rows = pl.read_parquet(ENRICHED_FILT, columns=["keyword"]).height
after_uks  = pl.read_parquet(ENRICHED_FILT, columns=["keyword"]).select(pl.col("keyword").n_unique()).item()

print(f"📊 Before filter: {before_rows:,} rows, {before_uks:,} unique keywords")
print(f"📊 After  filter: {after_rows:,} rows,  {after_uks:,} unique keywords  (should be close to {K:,})")

print(pl.read_parquet(ENRICHED_FILT, n_rows=8))


📥 Loading weekly_processed.parquet and computing keywords with ≥110 weeks…


/tmp/ipython-input-2153475164.py:49: DeprecationWarning: The argument `streaming=True` is deprecated and is being replaced by the `engine` argument.
  .collect(streaming=True)
/tmp/ipython-input-2153475164.py:49: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  .collect(streaming=True)


✅ Keywords with ≥110 weeks: 1,811
💾 Wrote index: /content/drive/MyDrive/colab_data/cleaned_cpu/keyword_index_110.csv
🧠 Loading model 'sentence-transformers/all-MiniLM-L6-v2' on cpu …
🔧 Computing embeddings …


Encoding keywords:   0%|          | 0/2 [00:00<?, ?it/s]

💾 Saved embeddings: /content/drive/MyDrive/colab_data/cleaned_cpu/keyword_embeddings_110.npy  shape=(1811, 384), dtype=float16
📐 Building full cosine similarity matrix …
💾 Saved similarity matrix: /content/drive/MyDrive/colab_data/cleaned_cpu/keyword_similarity_matrix_110.npy  shape=(1811, 1811), dtype=float16
🧮 Computing weekly weighted metrics …
Rows referencing the ≥110-weeks keyword set (sample touched): 1000


/tmp/ipython-input-2153475164.py:159: DeprecationWarning: The argument `streaming=True` is deprecated and is being replaced by the `engine` argument.
  .collect(streaming=True)
/tmp/ipython-input-2153475164.py:159: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  .collect(streaming=True)


Weeks:   0%|          | 0/127 [00:00<?, ?it/s]

/tmp/ipython-input-2153475164.py:231: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  metrics_df = pl.DataFrame(


💾 Wrote metrics: /content/drive/MyDrive/colab_data/cleaned_cpu/weekly_semantic_metrics_weighted_110.parquet
🔗 Attaching metrics to each original row (no aggregation)…
💾 Wrote enriched rows: /content/drive/MyDrive/colab_data/cleaned_cpu/weekly_rows_enriched_110.parquet
🧹 Filtering enriched rows to keywords in the new index…
📊 Before filter: 18,326,056 rows, 68,534 unique keywords
📊 After  filter: 10,451,327 rows,  1,811 unique keywords  (should be close to 1,811)
shape: (8, 11)
┌─────────┬────────────┬────────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ week    ┆ keyword    ┆ devicecate ┆ adcost   ┆ … ┆ avg_sim_t ┆ avg_sim_t ┆ n_sim_thi ┆ n_sim_las │
│ ---     ┆ ---        ┆ gory       ┆ ---      ┆   ┆ op25_this ┆ op25_last ┆ s_week    ┆ t_week    │
│ str     ┆ str        ┆ ---        ┆ f32      ┆   ┆ _week     ┆ _week     ┆ ---       ┆ ---       │
│         ┆            ┆ str        ┆          ┆   ┆ ---       ┆ ---       ┆ i32       ┆ i32       │
│         ┆  

# Embeddings, Similarity, and Weekly Semantic Metrics (≥110 weeks)

## Purpose
Compute sentence embeddings for long-horizon keywords (≥110 weeks), build a cosine similarity matrix, derive weekly semantic metrics, and attach them back to all rows, then filter the dataset to the ≥110-week keyword universe.

## Input
- `weekly_processed.parquet` (cleaned weekly rows with at least: week, keyword, devicecategory, adcost, adclicks, impressions, searchtype)

## Key Steps
1. Identify keywords with ≥110 distinct weeks.
2. Encode those keywords with `sentence-transformers/all-MiniLM-L6-v2` (on CPU) into normalized embeddings.
3. Build full cosine similarity matrix (cosine = dot due to normalization).
4. For each week (and previous week), compute top-25% neighbor weighted-average similarity per keyword.
5. Join metrics back to all original rows (no aggregation).
6. Filter rows down to the ≥110-week keyword universe.

## Sizes and Counts
- Keywords meeting threshold: **1,811**
- Embeddings saved: **shape (1811, 384), dtype float16**
- Cosine similarity matrix saved: **shape (1811, 1811), dtype float16**
- Weeks processed for metrics: **127**
- Before filter: **18,326,056 rows**, **68,534 unique keywords**
- After filter: **10,451,327 rows**, **1,811 unique keywords**
- Preview of enriched filtered output: **shape (8, 11)** shown

## Files Written
- `keyword_index_110.csv` — index of 1,811 keywords with their week counts.
- `keyword_embeddings_110.npy` — 384-dim float16 L2-normalized embeddings.
- `keyword_similarity_matrix_110.npy` — full cosine similarity matrix (float16).
- `weekly_semantic_metrics_weighted_110.parquet` — per (week, keyword) metrics:
  - `avg_sim_top25_this_week`, `avg_sim_top25_last_week`,
  - `n_sim_this_week`, `n_sim_last_week`.
- `weekly_rows_enriched_110.parquet` — original rows with metrics joined.
- `weekly_rows_enriched_filtered_110.parquet` — rows filtered to the 1,811-keyword universe.

## Notes
- Polars emitted deprecation warnings regarding `streaming=True`; functional impact is none for this run.
- A small sample of 1,000 rows was collected to touch the lazy pipeline before full computations.


# Aggregate Weekly Metrics by (week, keyword)

## Purpose
Summarize the enriched weekly dataset into one row per (week, keyword), computing totals and derived metrics, and creating count features for device category and search type.

## Input
- `/content/drive/MyDrive/colab_data/cleaned_cpu/weekly_rows_enriched_filtered_110.parquet`  
  Columns used: `week`, `keyword`, `devicecategory`, `searchtype`, `adcost`, `adclicks`, `impressions`, `avg_sim_top25_this_week`, `avg_sim_top25_last_week`, `n_sim_this_week`, `n_sim_last_week`.

## Transformations
1. **Type casting & selection** of relevant columns via Polars lazy scan.
2. **Aggregation by (week, keyword):**
   - `adcost_sum = sum(adcost)`
   - `adclicks_sum = sum(adclicks)`
   - `impressions_sum = sum(impressions)`
   - Carry forward first values of: `avg_sim_top25_this_week`, `avg_sim_top25_last_week`, `n_sim_this_week`, `n_sim_last_week`.
   - `cpc_week = adcost_sum / adclicks_sum` (only when `adclicks_sum > 0`).
3. **Category counts → wide format:**
   - Count rows by `devicecategory` and pivot to columns: `n_dev_<device>`.
   - Count rows by `searchtype` and pivot to columns: `n_st_<searchtype>`.
4. **Join** aggregates with pivoted count tables on `(week, keyword)` and **fill nulls with 0**.
5. **Column ordering**: base metrics first, then all additional count columns.

## Output
- `/content/drive/MyDrive/colab_data/cleaned_cpu/weekly_aggregated_by_week_keyword.parquet`  
  One row per (week, keyword) with aggregated metrics and category counts.

## Size and schema (from prior run)
- Shape: `(218,924, 15)`  
- Example columns:  
  `week`, `keyword`, `impressions_sum`, `cpc_week`,  
  `avg_sim_top25_this_week`, `avg_sim_top25_last_week`, `n_sim_this_week`, `n_sim_last_week`,  
  `n_dev_mobile`, `n_dev_desktop`, `n_dev_tablet`, `n_st_branded_search`, `n_st_generic_search`, …


In [ ]:
# !pip -q install polars pyarrow

from pathlib import Path
import re
import polars as pl

# -------- paths --------
BASE = Path("/content/drive/MyDrive/colab_data/cleaned_cpu")
IN_ROWS = BASE / "weekly_rows_enriched_filtered_110.parquet"
OUT_AGG = BASE / "weekly_aggregated_by_week_keyword.parquet"

if not IN_ROWS.exists():
    raise FileNotFoundError(IN_ROWS)

# -------- helper --------
def norm_suffix(s: str) -> str:
    s2 = str(s).lower().strip()
    s2 = re.sub(r"\s+", "_", s2)
    s2 = re.sub(r"[^a-z0-9_]+", "_", s2)
    s2 = re.sub(r"_+", "_", s2).strip("_")
    return s2 or "unknown"

# -------- scan main file --------
lf = pl.scan_parquet(str(IN_ROWS)).select(
    pl.col("week").cast(pl.Utf8),
    pl.col("keyword").cast(pl.Utf8),
    pl.col("devicecategory").cast(pl.Utf8, strict=False),
    pl.col("searchtype").cast(pl.Utf8, strict=False),
    pl.col("adcost").cast(pl.Float64, strict=False),
    pl.col("adclicks").cast(pl.Float64, strict=False),
    pl.col("impressions").cast(pl.Float64, strict=False),
    pl.col("avg_sim_top25_this_week").cast(pl.Float64, strict=False),
    pl.col("avg_sim_top25_last_week").cast(pl.Float64, strict=False),
    pl.col("n_sim_this_week").cast(pl.Int64, strict=False),
    pl.col("n_sim_last_week").cast(pl.Int64, strict=False),
)

# -------- aggregate base stats lazily --------
facts = (
    lf.group_by(["week", "keyword"])
      .agg(
          pl.col("adcost").sum().alias("adcost_sum"),
          pl.col("adclicks").sum().alias("adclicks_sum"),
          pl.col("impressions").sum().alias("impressions_sum"),
          pl.col("avg_sim_top25_this_week").first().alias("avg_sim_top25_this_week"),
          pl.col("avg_sim_top25_last_week").first().alias("avg_sim_top25_last_week"),
          pl.col("n_sim_this_week").first().alias("n_sim_this_week"),
          pl.col("n_sim_last_week").first().alias("n_sim_last_week"),
      )
      .with_columns(
          pl.when(pl.col("adclicks_sum") > 0)
            .then(pl.col("adcost_sum") / pl.col("adclicks_sum"))
            .otherwise(None)
            .alias("cpc_week")
      )
)

# -------- devicecategory & searchtype counts (eager, small) --------
df_dev = (
    lf.group_by(["week", "keyword", "devicecategory"])
      .agg(pl.len().alias("count"))
      .collect()
)
df_st = (
    lf.group_by(["week", "keyword", "searchtype"])
      .agg(pl.len().alias("count"))
      .collect()
)

# pivot device categories wide
if len(df_dev) > 0:
    df_dev_wide = (
        df_dev.pivot(index=["week", "keyword"], columns="devicecategory", values="count")
        .fill_null(0)
    )
    dev_cols = [c for c in df_dev_wide.columns if c not in ("week", "keyword")]
    df_dev_wide = df_dev_wide.rename({c: f"n_dev_{norm_suffix(c)}" for c in dev_cols})
else:
    df_dev_wide = pl.DataFrame({"week": [], "keyword": []})

# pivot search types wide
if len(df_st) > 0:
    df_st_wide = (
        df_st.pivot(index=["week", "keyword"], columns="searchtype", values="count")
        .fill_null(0)
    )
    st_cols = [c for c in df_st_wide.columns if c not in ("week", "keyword")]
    df_st_wide = df_st_wide.rename({c: f"n_st_{norm_suffix(c)}" for c in st_cols})
else:
    df_st_wide = pl.DataFrame({"week": [], "keyword": []})

# -------- collect lazy facts (large) --------
df_facts = facts.collect(streaming=True)

# -------- join all (eager) --------
df_agg = (
    df_facts.join(df_dev_wide, on=["week", "keyword"], how="left")
             .join(df_st_wide, on=["week", "keyword"], how="left")
             .fill_null(0)
)

# -------- reorder and save --------
base_cols = [
    "week", "keyword",
    "impressions_sum", "cpc_week",
    "avg_sim_top25_this_week", "avg_sim_top25_last_week",
    "n_sim_this_week", "n_sim_last_week",
]
extra_cols = [c for c in df_agg.columns if c not in base_cols]
df_out = df_agg.select(base_cols + sorted(extra_cols))

df_out.write_parquet(str(OUT_AGG), compression="zstd")
print(f"✅ Wrote aggregated file: {OUT_AGG}")
print(f"shape: {df_out.shape}")
print(df_out.head(10))


/tmp/ipython-input-3987014279.py:73: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  df_dev.pivot(index=["week", "keyword"], columns="devicecategory", values="count")
/tmp/ipython-input-3987014279.py:84: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  df_st.pivot(index=["week", "keyword"], columns="searchtype", values="count")
/tmp/ipython-input-3987014279.py:93: DeprecationWarning: The argument `streaming=True` is deprecated and is being replaced by the `engine` argument.
  df_facts = facts.collect(streaming=True)
/tmp/ipython-input-3987014279.py:93: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to 

✅ Wrote aggregated file: /content/drive/MyDrive/colab_data/cleaned_cpu/weekly_aggregated_by_week_keyword.parquet
shape: (218924, 15)
shape: (10, 15)
┌─────────┬────────────┬────────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ week    ┆ keyword    ┆ impression ┆ cpc_week ┆ … ┆ n_dev_mob ┆ n_dev_tab ┆ n_st_bran ┆ n_st_gene │
│ ---     ┆ ---        ┆ s_sum      ┆ ---      ┆   ┆ ile       ┆ let       ┆ ded_searc ┆ ric_searc │
│ str     ┆ str        ┆ ---        ┆ f64      ┆   ┆ ---       ┆ ---       ┆ h         ┆ h         │
│         ┆            ┆ f64        ┆          ┆   ┆ u32       ┆ u32       ┆ ---       ┆ ---       │
│         ┆            ┆            ┆          ┆   ┆           ┆           ┆ u32       ┆ u32       │
╞═════════╪════════════╪════════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 02-2021 ┆ car rental ┆ 18.0       ┆ 1.016667 ┆ … ┆ 9         ┆ 0         ┆ 0         ┆ 10        │
│         ┆ airport    ┆            ┆      

# Output Interpretation: weekly_aggregated_by_week_keyword.parquet

## Overview
This output file summarizes all keyword-level advertising activity on a **weekly basis**.  
Each row represents one `(week, keyword)` combination, containing aggregated ad performance metrics, semantic similarity measures, and distribution counts by device and search type.

## Output Summary
- **File written:** `/content/drive/MyDrive/colab_data/cleaned_cpu/weekly_aggregated_by_week_keyword.parquet`
- **Shape:** (218,924 rows, 15 columns)
- **Schema highlights:**
  - `week`: ISO week (WW-YYYY)
  - `keyword`: normalized keyword string
  - `impressions_sum`: total ad impressions in that week
  - `adcost_sum`, `adclicks_sum`: total costs and clicks (intermediate)
  - `cpc_week`: average cost per click for that week (`adcost_sum / adclicks_sum`)
  - `avg_sim_top25_this_week` / `avg_sim_top25_last_week`: weighted average of top-25% semantic similarity among neighbors
  - `n_sim_this_week` / `n_sim_last_week`: number of keywords used in similarity weighting
  - `n_dev_*`: counts of observations by device category (e.g. `n_dev_mobile`, `n_dev_tablet`)
  - `n_st_*`: counts of observations by search type (e.g. `n_st_branded_search`, `n_st_generic_search`)

## Sample Rows (excerpt)
| week    | keyword            | impressions_sum | cpc_week | n_dev_mobile | n_dev_tablet | n_st_branded_search | n_st_generic_search |
|----------|--------------------|-----------------|-----------|---------------|---------------|----------------------|----------------------|
| 02-2021 | car rental airport sixt chicago | 18.0 | 1.02 | 9 | 0 | 0 | 10 |
| 47-2021 | car rental maui exotic | 39.0 | 1.67 | 2 | 0 | 0 | 7 |
| 22-2023 | car rental green motion orlando | 48.0 | 1.18 | 5 | 0 | 0 | 6 |
| 40-2021 | car rental tempest | 70.0 | 0.65 | 13 | 1 | 22 | 0 |
| 14-2021 | budget carhire | 8.0 | 1.29 | 6 | 0 | 8 | 0 |




## Including all exogenous columns

In [ ]:
# Remove all contents from the mount point before attempting to mount
!rm -rf /content/drive/*
print("Cleared /content/drive directory.")

# Then, mount Google Drive with force_remount=True
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Cleared /content/drive directory.
Mounted at /content/drive


# Per-Keyword Export with Neighbors’ Last-Week CPC (Full Feature Tables)

## Purpose
Create one Parquet file **per keyword** that:
1. Keeps **all columns** from the aggregated weekly dataset for that keyword, and
2. Adds **Top-5 neighbors’ CPC at the previous week** as extra columns:
   - `cpc_lastweek__<neighbor_1>`, …, `cpc_lastweek__<neighbor_5>`

## Inputs
- `.../cleaned_cpu/weekly_aggregated_by_week_keyword.parquet`  
  (base weekly table with `week`, `keyword`, `cpc_week`, and all exogenous features)
- `.../cleaned_cpu/keyword_index_110.csv`  
  (keyword universe aligned with the similarity matrix; columns: `index`, `keyword`, `n_weeks`)
- `.../cleaned_cpu/keyword_similarity_matrix_110.npy`  
  (K×K cosine similarity; same order as the CSV)

## Transformations
1. **Load keyword index and similarity matrix** and assert shape alignment.
2. **For each keyword**, find **Top-5** most similar neighbors (excluding itself).
3. **Normalize week labels** to `WW-YYYY` (defensive).
4. **Compute previous week** for each row: `iso_week_prev(week)` (handles year wrap).
5. **Join neighbors’ CPC**: for each neighbor, left-join that neighbor’s `cpc_week` on the current keyword’s `week_prev`.
6. **Select columns**:
   - All original columns except the helper `week_prev`,
   - Plus the five neighbor lag columns `cpc_lastweek__*`.

## Outputs
- Directory: `.../cleaned_cpu/keywords_dfs_full/`
- Files: one Parquet per keyword, filename is **slugified** keyword (ASCII, lowercase, `_`).
- Each file contains:
  - Original columns from `weekly_aggregated_by_week_keyword.parquet` for that keyword (e.g., `week`, `keyword`, `cpc_week`, device/search counts, similarity metrics, etc.),
  - Neighbor lag features: `cpc_lastweek__<neighbor_i>` for i = 1..5.

## Notes
- If a keyword has **no rows** in the aggregated dataset, an **empty file** with `week` and `keyword` schema is written for consistency - will not happen because 110 weeks are existent for every keyword
- No existing per-keyword files are required; this step **creates them from scratch**.


In [ ]:
!pip -q install polars pyarrow tqdm numpy unidecode

import os, re, numpy as np, polars as pl
from tqdm import tqdm
from datetime import datetime, timedelta
from unidecode import unidecode

# ---------- Paths ----------
BASE_DIR = "/content/drive/MyDrive/colab_data"
AGG_PARQUET = f"{BASE_DIR}/cleaned_cpu/weekly_aggregated_by_week_keyword.parquet"
KW_INDEX_CSV = f"{BASE_DIR}/cleaned_cpu/keyword_index_110.csv"
SIM_NPY = f"{BASE_DIR}/cleaned_cpu/keyword_similarity_matrix_110.npy"
OUT_DIR = f"{BASE_DIR}/cleaned_cpu/keywords_dfs_full"

os.makedirs(OUT_DIR, exist_ok=True)

# ---------- Helpers ----------
def iso_week_prev(wwyyyy: str) -> str:
    """Return previous ISO week label 'WW-YYYY' (wraps year correctly)."""
    if wwyyyy is None:
        return None
    ww, yyyy = wwyyyy.split("-")
    week = int(ww)
    year = int(yyyy)
    base = datetime.fromisocalendar(year, week, 1)
    prev = base - timedelta(days=7)
    py, pw, _ = prev.isocalendar()
    return f"{pw:02d}-{py}"

_slug_re = re.compile(r"[^a-z0-9_]+")
def slugify(s: str) -> str:
    s = unidecode(s.lower().strip().replace(" ", "_"))
    s = _slug_re.sub("_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or "keyword"

# ---------- Load index & similarity ----------
kw_idx = pl.read_csv(KW_INDEX_CSV)  # expects columns: index, keyword, n_weeks
kw_idx = kw_idx.sort("index")
keywords = kw_idx["keyword"].to_list()

print("Loaded keyword index:")
print(kw_idx.head())
print(f"Total keywords in index: {len(keywords)}")

sim = np.load(SIM_NPY)  # shape (K,K), float16
K = sim.shape[0]
print("Similarity matrix shape:", sim.shape)
assert K == len(keywords), "Keyword index and similarity matrix shape mismatch."

# Precompute top-5 neighbor indices for each keyword (exclude itself)
topN = 5
neighbor_idxs = []
for i in range(K):
    row = sim[i].astype(np.float32)
    row[i] = -np.inf
    cand = np.argpartition(-row, range(topN))[:topN]
    cand = cand[np.argsort(-row[cand])]
    neighbor_idxs.append(cand.tolist())

print("Example neighbors for first keyword:")
print("  keyword:", keywords[0])
print("  neighbors:", [keywords[j] for j in neighbor_idxs[0]])

# ---------- Load base aggregated dataset ----------
print("\nReading aggregated parquet:", AGG_PARQUET)
base_df = pl.read_parquet(AGG_PARQUET)
print("Base DF shape:", base_df.shape)
print("Base DF columns:", base_df.columns)
print("Base DF head:")
print(base_df.head())

# Optional: ensure week is in 'WW-YYYY' zero-padded format (defensive)
def _normalize_week(df: pl.DataFrame) -> pl.DataFrame:
    # cast week to string first to be safe
    df = df.with_columns(pl.col("week").cast(pl.Utf8))
    return df.with_columns(
        pl.when(pl.col("week").str.contains(r"^\d{2}-\d{4}$"))
        .then(pl.col("week"))
        .otherwise(
            pl.col("week").map_elements(
                lambda s: (
                    f"{int(s.split('-')[0]):02d}-{int(s.split('-')[1]):04d}"
                    if (s is not None and isinstance(s, str) and "-" in s)
                    else s
                ),
                return_dtype=pl.Utf8
            )
        ).alias("week")
    )

base_df = _normalize_week(base_df)

print("\nAfter week normalization:")
print("Unique week examples:", base_df["week"].unique().sort().head(10))
print("Base DF head (week/keyword/cpc_week):")
print(base_df.select(["week", "keyword", "cpc_week"]).head())

# ---------- Per-keyword export ----------
for i in tqdm(range(K), desc="Per-keyword exports"):
    kw = keywords[i]
    neighbors = [keywords[j] for j in neighbor_idxs[i]]



    # rows for target keyword (ALL columns to keep all exogenous vars)
    df_k = base_df.filter(pl.col("keyword") == kw)



    if df_k.is_empty():
        if i < 3 or kw == "car rental heathrow terminal 2":
            print(">>> df_k is EMPTY. Writing empty file.")
        out_path = os.path.join(OUT_DIR, f"{slugify(kw)}.parquet")
        # write an empty DF with schema = week, keyword only to keep it simple
        pl.DataFrame(schema={"week": pl.Utf8, "keyword": pl.Utf8}).write_parquet(
            out_path, compression="zstd"
        )
        continue

    # compute previous week label for alignment with neighbors
    df_k = df_k.with_columns(
        pl.col("week").map_elements(iso_week_prev, return_dtype=pl.Utf8).alias("week_prev")
    )



    # add each neighbor's last-week CPC via left-join on week_prev
    df_aug = df_k
    for nb in neighbors:
        colname = f"cpc_lastweek__{nb}"

        nb_df = base_df.filter(pl.col("keyword") == nb).select(
            pl.col("week").alias("week_prev"),
            pl.col("cpc_week").alias(colname),
        )


        df_aug = df_aug.join(nb_df, on="week_prev", how="left")

    # final tidy: keep week, keyword, own cpc_week, and neighbor columns (drop week_prev)
    neighbor_cols = [c for c in df_aug.columns if c.startswith("cpc_lastweek__")]
    keep_cols = (
        ["week", "keyword", "cpc_week"]
        + [c for c in df_aug.columns if c not in ["week", "keyword", "cpc_week", "week_prev"] and not c.startswith("cpc_lastweek__")]
        + neighbor_cols
    )

    df_out = df_aug.select(keep_cols)



    # write file
    out_path = os.path.join(OUT_DIR, f"{slugify(kw)}.parquet")
    df_out.write_parquet(out_path, compression="zstd")


Loaded keyword index:
shape: (5, 3)
┌───────┬───────────────┬─────────┐
│ index ┆ keyword       ┆ n_weeks │
│ ---   ┆ ---           ┆ ---     │
│ i64   ┆ str           ┆ i64     │
╞═══════╪═══════════════╪═════════╡
│ 0     ┆ 24 hour rent  ┆ 125     │
│ 1     ┆ 3 month hire  ┆ 119     │
│ 2     ┆ 7 seater hire ┆ 122     │
│ 3     ┆ aberdeen hire ┆ 122     │
│ 4     ┆ aco rent      ┆ 126     │
└───────┴───────────────┴─────────┘
Total keywords in index: 1811
Similarity matrix shape: (1811, 1811)
Example neighbors for first keyword:
  keyword: 24 hour rent
  neighbors: ['car rental 24 hour', 'rent', 'rent place', 'rent for month', 'asap rent']

Reading aggregated parquet: /content/drive/MyDrive/colab_data/cleaned_cpu/weekly_aggregated_by_week_keyword.parquet
Base DF shape: (218924, 15)
Base DF columns: ['week', 'keyword', 'impressions_sum', 'cpc_week', 'avg_sim_top25_this_week', 'avg_sim_top25_last_week', 'n_sim_this_week', 'n_sim_last_week', 'adclicks_sum', 'adcost_sum', 'n_dev_desktop'

Per-keyword exports: 100%|██████████| 1811/1811 [01:23<00:00, 21.79it/s]


In [ ]:
import os, random, polars as pl

OUT_DIR = "/content/drive/MyDrive/colab_data/cleaned_cpu/keywords_dfs_full"

files = [f for f in os.listdir(OUT_DIR) if f.endswith(".parquet")]
print("Number of files:", len(files))

sample_file = random.choice(files)
print("Random file:", sample_file)

df = pl.read_parquet(os.path.join(OUT_DIR, sample_file))
print(df.shape)
print(df.head())


Number of files: 1811
Random file: car_rental_north_miami.parquet
(115, 20)
shape: (5, 20)
┌─────────┬────────────┬──────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ week    ┆ keyword    ┆ cpc_week ┆ impression ┆ … ┆ cpc_lastw ┆ cpc_lastw ┆ cpc_lastw ┆ cpc_lastw │
│ ---     ┆ ---        ┆ ---      ┆ s_sum      ┆   ┆ eek__car  ┆ eek__car  ┆ eek__car  ┆ eek__car  │
│ str     ┆ str        ┆ f64      ┆ ---        ┆   ┆ rental    ┆ rental    ┆ rental    ┆ rental    │
│         ┆            ┆          ┆ f64        ┆   ┆ sixt …    ┆ miami…    ┆ downt…    ┆ miami…    │
│         ┆            ┆          ┆            ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│         ┆            ┆          ┆            ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞═════════╪════════════╪══════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 35-2021 ┆ car rental ┆ 1.763333 ┆ 24.0       ┆ … ┆ 1.076     ┆ 2.14      ┆ null      ┆ 6.89      │


## Create Varying number of similar Cpcs per keyword

In [ ]:
!pip -q install polars pyarrow tqdm numpy unidecode

import os, re, numpy as np, polars as pl
from tqdm import tqdm
from datetime import datetime, timedelta
from unidecode import unidecode

# ---------- Config ----------
BASE_DIR = "/content/drive/MyDrive/colab_data"
AGG_PARQUET = f"{BASE_DIR}/cleaned_cpu/weekly_aggregated_by_week_keyword.parquet"
KW_INDEX_CSV = f"{BASE_DIR}/cleaned_cpu/keyword_index_110.csv"
SIM_NPY = f"{BASE_DIR}/cleaned_cpu/keyword_similarity_matrix_110.npy"

# how many similar neighbors you want variants for
N_LIST = [2, 3, 5, 7, 10, 20]
MAX_NEIGHBORS = max(N_LIST)

# output dirs for each N
OUT_DIRS = {
    N: f"{BASE_DIR}/cleaned_cpu/keywords_dfs_full_{N}"
    for N in N_LIST
}
for d in OUT_DIRS.values():
    os.makedirs(d, exist_ok=True)

# ---------- Helpers ----------
def iso_week_prev(wwyyyy: str) -> str:
    """Return previous ISO week label 'WW-YYYY' (wraps year correctly)."""
    if wwyyyy is None:
        return None
    ww, yyyy = wwyyyy.split("-")
    week = int(ww)
    year = int(yyyy)
    base = datetime.fromisocalendar(year, week, 1)
    prev = base - timedelta(days=7)
    py, pw, _ = prev.isocalendar()
    return f"{pw:02d}-{py}"

_slug_re = re.compile(r"[^a-z0-9_]+")
def slugify(s: str) -> str:
    s = unidecode(s.lower().strip().replace(" ", "_"))
    s = _slug_re.sub("_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or "keyword"

# ---------- Load index & similarity ----------
kw_idx = pl.read_csv(KW_INDEX_CSV).sort("index")  # expects: index, keyword, n_weeks
keywords = kw_idx["keyword"].to_list()

sim = np.load(SIM_NPY)  # shape (K,K), float16
K = sim.shape[0]
assert K == len(keywords), "Keyword index and similarity matrix shape mismatch."

# Precompute top MAX_NEIGHBORS neighbor indices for each keyword (exclude itself)
neighbor_idxs = []
for i in range(K):
    row = sim[i].astype(np.float32)
    row[i] = -np.inf
    cand = np.argpartition(-row, range(MAX_NEIGHBORS))[:MAX_NEIGHBORS]
    cand = cand[np.argsort(-row[cand])]  # sort by similarity desc
    neighbor_idxs.append(cand.tolist())

# ---------- Load base aggregated dataset ----------
base_df = pl.read_parquet(AGG_PARQUET)

def _normalize_week(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns(pl.col("week").cast(pl.Utf8))
    return df.with_columns(
        pl.when(pl.col("week").str.contains(r"^\d{2}-\d{4}$"))
        .then(pl.col("week"))
        .otherwise(
            pl.col("week").map_elements(
                lambda s: (
                    f"{int(s.split('-')[0]):02d}-{int(s.split('-')[1]):04d}"
                    if (s is not None and isinstance(s, str) and "-" in s)
                    else s
                ),
                return_dtype=pl.Utf8
            )
        ).alias("week")
    )

base_df = _normalize_week(base_df)

print("Base DF shape:", base_df.shape)
print("Columns:", base_df.columns)

# ---------- Per-keyword export with multiple N ----------
for i in tqdm(range(K), desc="Per-keyword exports (multi-N)"):
    kw = keywords[i]
    neighbors_full = [keywords[j] for j in neighbor_idxs[i]]  # length MAX_NEIGHBORS

    # rows for target keyword (all exogenous vars)
    df_k = base_df.filter(pl.col("keyword") == kw)

    if df_k.is_empty():
        # no data for this keyword: write an empty file per N with simple schema
        for N in N_LIST:
            out_path = os.path.join(OUT_DIRS[N], f"{slugify(kw)}.parquet")
            pl.DataFrame(schema={"week": pl.Utf8, "keyword": pl.Utf8}).write_parquet(
                out_path, compression="zstd"
            )
        continue

    # compute previous week for this keyword
    df_k = df_k.with_columns(
        pl.col("week").map_elements(iso_week_prev, return_dtype=pl.Utf8).alias("week_prev")
    )

    # join ALL MAX_NEIGHBORS neighbors' last-week CPCs
    df_aug = df_k
    for nb in neighbors_full:
        colname = f"cpc_lastweek__{nb}"
        nb_df = base_df.filter(pl.col("keyword") == nb).select(
            pl.col("week").alias("week_prev"),
            pl.col("cpc_week").alias(colname),
        )
        df_aug = df_aug.join(nb_df, on="week_prev", how="left")

    # figure out exogenous columns vs neighbor columns once
    neighbor_cols_all = [c for c in df_aug.columns if c.startswith("cpc_lastweek__")]
    exog_cols = [c for c in df_aug.columns if c not in (["week_prev"] + neighbor_cols_all)]

    # now create one DF per N by selecting only first N neighbor CPC columns
    # (neighbors_full is sorted by similarity, and we joined in that order)
    for N in N_LIST:
        cols_N = neighbor_cols_all[:N]
        df_out = df_aug.select(exog_cols + cols_N)
        out_path = os.path.join(OUT_DIRS[N], f"{slugify(kw)}.parquet")
        df_out.write_parquet(out_path, compression="zstd")


Base DF shape: (218924, 15)
Columns: ['week', 'keyword', 'impressions_sum', 'cpc_week', 'avg_sim_top25_this_week', 'avg_sim_top25_last_week', 'n_sim_this_week', 'n_sim_last_week', 'adclicks_sum', 'adcost_sum', 'n_dev_desktop', 'n_dev_mobile', 'n_dev_tablet', 'n_st_branded_search', 'n_st_generic_search']


Per-keyword exports (multi-N): 100%|██████████| 1811/1811 [06:06<00:00,  4.95it/s]


In [ ]:
import os, random
import polars as pl

BASE_DIR = "/content/drive/MyDrive/colab_data/cleaned_cpu"
folders = [
    f"{BASE_DIR}/keywords_dfs_full_2",
    f"{BASE_DIR}/keywords_dfs_full_3",
    f"{BASE_DIR}/keywords_dfs_full_5",
    f"{BASE_DIR}/keywords_dfs_full_7",
    f"{BASE_DIR}/keywords_dfs_full_10",
    f"{BASE_DIR}/keywords_dfs_full_20",
]

print("=== Inspecting one random file per folder ===\n")

for folder in folders:
    print(f"Folder: {folder}")
    files = [f for f in os.listdir(folder) if f.endswith(".parquet")]

    if not files:
        print("  → No files found.\n")
        continue

    sample = random.choice(files)
    print(f"  Random file: {sample}")

    df = pl.read_parquet(os.path.join(folder, sample))

    print(f"  Shape: {df.shape}")
    print(df.head(5))
    print("\n" + "-"*80 + "\n")


=== Inspecting one random file per folder ===

Folder: /content/drive/MyDrive/colab_data/cleaned_cpu/keywords_dfs_full_2
  Random file: city_rent.parquet
  Shape: (125, 17)
shape: (5, 17)
┌─────────┬───────────┬────────────┬──────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ week    ┆ keyword   ┆ impression ┆ cpc_week ┆ … ┆ n_st_brand ┆ n_st_gene ┆ cpc_lastw ┆ cpc_lastw │
│ ---     ┆ ---       ┆ s_sum      ┆ ---      ┆   ┆ ed_search  ┆ ric_searc ┆ eek__rent ┆ eek__rent │
│ str     ┆ str       ┆ ---        ┆ f64      ┆   ┆ ---        ┆ h         ┆ ---       ┆ place     │
│         ┆           ┆ f64        ┆          ┆   ┆ u32        ┆ ---       ┆ f64       ┆ ---       │
│         ┆           ┆            ┆          ┆   ┆            ┆ u32       ┆           ┆ f64       │
╞═════════╪═══════════╪════════════╪══════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 35-2021 ┆ city rent ┆ 36.0       ┆ 0.647246 ┆ … ┆ 9          ┆ 0         ┆ 2.606408  ┆ 2.771111  │
│ 31

## current status all below deprecated for local run

In [ ]:
# One self-contained cell: Polars-only (CPU), vectorized keyword normalization

# 1) Setup
import os, glob, sys, subprocess

DATA_DIR = "/content/drive/MyDrive/colab_data"
OUT_DIR  = os.path.join(DATA_DIR, "cleaned_cpu")
os.makedirs(OUT_DIR, exist_ok=True)

FILES = sorted(glob.glob(os.path.join(DATA_DIR, "part.*.parquet")))
print(f"Found {len(FILES)} parquet parts in {DATA_DIR}")

# 2) Install deps (quiet)
def _pip_install(pkgs):
    return subprocess.run([sys.executable, "-m", "pip", "install", "-q", *pkgs], check=False).returncode == 0

_pip_install(["polars>=1.0.0", "pyarrow"])

import polars as pl

# 3) Vectorized transform that mirrors your unify_keyword() exactly
def pl_unify_keyword(expr: pl.Expr) -> pl.Expr:
    s = expr.cast(pl.Utf8).str.to_lowercase()

    # Replace ".com" and punctuation . - + " ' with space
    s = s.str.replace_all(r"\.com|[.\-\+\"']", " ")

    # Remove standalone 'in' and 'a'
    s = s.str.replace_all(r"\b(in|a)\b", "")

    # Normalize spaces and trim
    s = s.str.replace_all(r"\s+", " ").str.strip_chars()

    # Presence flags
    has_car     = s.str.contains(r"\bcar\b")
    has_rental  = s.str.contains(r"\brental\b")
    has_airport = s.str.contains(r"\bairport\b")

    # Remove these tokens from the tail
    tail = s.str.replace_all(r"\b(car|rental|airport)\b", "")
    tail = tail.str.replace_all(r"\s+", " ").str.strip_chars()

    # Build prefix: 'car rental' if both present, then 'airport' if present
    prefix = pl.when(has_car & has_rental).then(pl.lit("car rental")).otherwise(pl.lit(""))
    prefix = (prefix + pl.lit(" ") + pl.when(has_airport).then(pl.lit("airport")).otherwise(pl.lit("")))
    prefix = prefix.str.replace_all(r"\s+", " ").str.strip_chars()

    # Combine prefix + tail, cleanup
    out = (prefix + pl.lit(" ") + tail).str.replace_all(r"\s+", " ").str.strip_chars()
    return out

# 4) Process all files
for fp in FILES:
    name = os.path.basename(fp)
    print(f"CPU Processing: {name}")
    df = pl.read_parquet(fp)
    if "keyword" not in df.columns:
        print("  -> Skipped (no 'keyword' column).")
        continue

    df = df.with_columns(keyword = pl_unify_keyword(pl.col("keyword")))
    out_fp = os.path.join(OUT_DIR, name)
    df.write_parquet(out_fp, compression="snappy")
    print(f"  -> Saved: {out_fp}")

print(f"Done. Cleaned files in: {OUT_DIR}")


Streaming output truncated to the last 5000 lines.
  -> Saved: /content/drive/MyDrive/colab_data/cleaned_cpu/part.2717.parquet
CPU Processing: part.2718.parquet
  -> Saved: /content/drive/MyDrive/colab_data/cleaned_cpu/part.2718.parquet
CPU Processing: part.2719.parquet
  -> Saved: /content/drive/MyDrive/colab_data/cleaned_cpu/part.2719.parquet
CPU Processing: part.272.parquet
  -> Saved: /content/drive/MyDrive/colab_data/cleaned_cpu/part.272.parquet
CPU Processing: part.2720.parquet
  -> Saved: /content/drive/MyDrive/colab_data/cleaned_cpu/part.2720.parquet
CPU Processing: part.2721.parquet
  -> Saved: /content/drive/MyDrive/colab_data/cleaned_cpu/part.2721.parquet
CPU Processing: part.2722.parquet
  -> Saved: /content/drive/MyDrive/colab_data/cleaned_cpu/part.2722.parquet
CPU Processing: part.2723.parquet
  -> Saved: /content/drive/MyDrive/colab_data/cleaned_cpu/part.2723.parquet
CPU Processing: part.2724.parquet
  -> Saved: /content/drive/MyDrive/colab_data/cleaned_cpu/part.2724.par

In [ ]:
# !pip install -q polars pyarrow

from pathlib import Path
import polars as pl

DATA_DIR = Path("/content/drive/MyDrive/colab_data/cleaned_cpu")
OUT_PARQUET = DATA_DIR / "weekly_processed.parquet"

DROP_COLS = {"admatchedquery", "addestinationurl", "exchangerateusd", "cost_per_click", "domain"}
KEEP_COLS_BASE = ["keyword", "devicecategory", "adcost", "adclicks", "impressions", "searchtype"]

# --- Load parquet files ---
paths = [str(p) for p in DATA_DIR.rglob("*.parquet")
         if "__MACOSX" not in str(p) and not p.name.startswith(("._", "."))]
if not paths:
    raise FileNotFoundError(f"No parquet files found under {DATA_DIR}")

print(f"Found {len(paths)} parquet files. Building lazy scan…")
lf = pl.scan_parquet(paths, allow_missing_columns=True)

names = set(lf.collect_schema().names())
for req in ("date", "keyword"):
    if req not in names:
        raise ValueError(f"Missing required '{req}'. Found: {sorted(names)}")

present_keep = [c for c in KEEP_COLS_BASE if c in names]

# --- Select minimal columns ---
lf_sel = lf.select(
    pl.col("date"),
    pl.col("keyword").cast(pl.Utf8).alias("keyword"),
    *[pl.col(c) for c in present_keep if c != "keyword"]
)

# --- Robust date parsing (handles string + datetime) ---
date_str = pl.col("date").cast(pl.Utf8, strict=False)

fmt_attempts = [
    "%Y-%m-%d",
    "%Y/%m/%d",
    "%d/%m/%Y",
    "%m/%d/%Y",
    "%Y-%m-%d %H:%M:%S",
    "%Y/%m/%d %H:%M:%S",
    "%Y-%m-%dT%H:%M:%S",
    "%Y-%m-%dT%H:%M:%S%.f",
]

parsed_attempts = [date_str.str.strptime(pl.Date, format=f, strict=False) for f in fmt_attempts]
parsed_attempts += [
    pl.col("date").cast(pl.Date, strict=False),
    pl.col("date").cast(pl.Datetime, strict=False).dt.date(),
]

parsed_date = pl.coalesce(parsed_attempts).alias("date_parsed")

# --- Preflight check: print bad dates if any ---
print("🔎 Preflight: validating date parsing on a sample…")
test = (
    lf_sel
    .with_columns(parsed_date)
    .select(pl.col("date"), pl.col("date_parsed"))
    .filter(pl.col("date_parsed").is_null())
    .limit(20)
    .collect(streaming=True)
)

if test.height > 0:
    print("❌ Could not parse these 'date' values (showing up to 20):")
    print(test)
    raise ValueError(
        "Date parsing failed for some rows. Check printed values and add a matching format if needed."
    )
else:
    print("✅ Preflight OK: sample parsed successfully.")

# --- Transform and write ---
lf_clean = (
    lf_sel
    .with_columns(parsed_date)
    .drop("date")
    .with_columns(
        pl.concat_str([
            pl.col("date_parsed").dt.strftime("%V"),
            pl.lit("-"),
            pl.col("date_parsed").dt.strftime("%G"),
        ]).alias("week")
    )
    .drop("date_parsed")
    .drop(list(DROP_COLS), strict=False)
    .select("week", "keyword", *[c for c in present_keep if c != "keyword"])
)

print("🧮 Writing weekly_processed.parquet (compressed, streaming)…")
lf_clean.sink_parquet(
    str(OUT_PARQUET),
    compression="zstd",
    statistics=True,
    row_group_size=8192
)

print(f"✅ Done. Wrote: {OUT_PARQUET}")
print(pl.read_parquet(OUT_PARQUET, n_rows=10))


Found 4410 parquet files. Building lazy scan…
🔎 Preflight: validating date parsing on a sample…


/tmp/ipython-input-3128736239.py:65: DeprecationWarning: The argument `streaming=True` is deprecated and is being replaced by the `engine` argument.
  .collect(streaming=True)
/tmp/ipython-input-3128736239.py:65: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  .collect(streaming=True)


✅ Preflight OK: sample parsed successfully.
🧮 Writing weekly_processed.parquet (compressed, streaming)…
✅ Done. Wrote: /content/drive/MyDrive/colab_data/cleaned_cpu/weekly_processed.parquet
shape: (10, 7)
┌─────────┬────────────────────┬────────────────┬────────┬──────────┬─────────────┬────────────────┐
│ week    ┆ keyword            ┆ devicecategory ┆ adcost ┆ adclicks ┆ impressions ┆ searchtype     │
│ ---     ┆ ---                ┆ ---            ┆ ---    ┆ ---      ┆ ---         ┆ ---            │
│ str     ┆ str                ┆ str            ┆ f32    ┆ f64      ┆ f64         ┆ str            │
╞═════════╪════════════════════╪════════════════╪════════╪══════════╪═════════════╪════════════════╡
│ 01-2021 ┆ rent               ┆ desktop        ┆ 1.91   ┆ 1.0      ┆ 1.0         ┆ Branded Search │
│ 02-2021 ┆ car rental avis    ┆ desktop        ┆ 1.59   ┆ 1.0      ┆ 2.0         ┆ Generic Search │
│ 02-2021 ┆ budget rentals     ┆ desktop        ┆ 15.31  ┆ 2.0      ┆ 25.0        ┆ Gene

In [ ]:
# === Semantic embeddings (fast, Colab Pro L4) + optional full similarity matrix ===
# Input : /content/drive/MyDrive/colab_data/weekly_processed.parquet  (needs: keyword, week)
# Output: keyword_index.csv, keyword_embeddings.npy, and (optional) keyword_similarity_matrix.npy/csv

# !pip -q install polars pyarrow sentence-transformers tqdm

from pathlib import Path
import os, gc, math
import numpy as np
import polars as pl
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

# ---------------- CONFIG ----------------
DATA_DIR = Path("/content/drive/MyDrive/colab_data/cleaned_cpu")
IN_PARQUET = DATA_DIR / "weekly_processed.parquet"

MIN_WEEKS = 110

# Models:
#   Fast English:       "sentence-transformers/all-MiniLM-L6-v2"
#   Fast Multilingual:  "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

BATCH_SIZE = 1024        # increase if plenty of VRAM, decrease if OOM
EMB_DTYPE = np.float16   # store embeddings as fp16 to halve disk/VRAM
WANT_MATRIX = True       # set False if you only want embeddings now (fastest)
BLOCK = 4096             # tile size for GPU matmul if building the matrix
CSV_IF_K_LE = 3000       # also write CSV matrix if K is small
# ---------------------------------------

OUT_IDX = DATA_DIR / "keyword_index.csv"
OUT_EMB = DATA_DIR / "keyword_embeddings.npy"
OUT_SIM = DATA_DIR / "keyword_similarity_matrix.npy"
OUT_CSV = DATA_DIR / "keyword_similarity_matrix.csv"

if not IN_PARQUET.exists():
    raise FileNotFoundError(f"Input parquet not found: {IN_PARQUET}")

# ---------- 1) Load & filter keywords ----------
print("📥 Loading cleaned data…")
df = pl.scan_parquet(str(IN_PARQUET))

for req in ("keyword", "week"):
    if req not in df.collect_schema().names():
        raise ValueError(f"Required column '{req}' missing.")

print(f"🔎 Keeping keywords with ≥{MIN_WEEKS} distinct weeks…")
kw_weeks = (
    df.select(pl.col("keyword").cast(pl.Utf8), pl.col("week").cast(pl.Utf8))
      .unique(subset=["keyword", "week"])
      .group_by("keyword")
      .agg(pl.n_unique("week").alias("n_weeks"))
      .filter(pl.col("n_weeks") >= MIN_WEEKS)
      .sort("keyword")
      .collect(streaming=True)
)

K = kw_weeks.height
if K == 0:
    raise RuntimeError(f"No keywords appear in ≥{MIN_WEEKS} weeks.")
keywords = kw_weeks["keyword"].to_list()
n_weeks_vec = kw_weeks["n_weeks"].to_list()
print(f"✅ Kept {K:,} keywords.")

# Save index mapping now
idx_df = pd.DataFrame({"index": range(K), "keyword": keywords, "n_weeks": n_weeks_vec})
idx_df.to_csv(OUT_IDX, index=False)
print(f"💾 Wrote index mapping: {OUT_IDX}")

# ---------- 2) Encode on GPU ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🧠 Loading model '{MODEL_NAME}' on {device}…")
model = SentenceTransformer(MODEL_NAME, device=device)

# Speed hints for newer GPUs
torch.backends.cudnn.benchmark = True
if hasattr(torch, "set_float32_matmul_precision"):
    torch.set_float32_matmul_precision("high")

use_amp = (device == "cuda")
amp_dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float16

def embed_texts(texts, batch_size=BATCH_SIZE):
    out_list = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Encoding keywords"):
        batch = texts[i:i+batch_size]
        with torch.inference_mode():
            if use_amp:
                with torch.autocast(device_type="cuda", dtype=amp_dtype):
                    e = model.encode(
                        batch,
                        batch_size=len(batch),
                        convert_to_numpy=True,
                        show_progress_bar=False,
                        normalize_embeddings=True,  # L2-normalized => dot = cosine
                    )
            else:
                e = model.encode(
                    batch,
                    batch_size=len(batch),
                    convert_to_numpy=True,
                    show_progress_bar=False,
                    normalize_embeddings=True,
                )
        out_list.append(e.astype(np.float32, copy=False))
    return np.vstack(out_list)

print("🔧 Computing embeddings…")
emb_fp32 = embed_texts(keywords, batch_size=BATCH_SIZE)   # (K, d)
d = emb_fp32.shape[1]
emb_fp16 = emb_fp32.astype(EMB_DTYPE, copy=False)
del emb_fp32; gc.collect()

# Save embeddings
np.save(OUT_EMB, emb_fp16)
print(f"💾 Saved embeddings: {OUT_EMB}  (shape {emb_fp16.shape}, dtype={emb_fp16.dtype})")

# Quick early exit if only embeddings desired
if not WANT_MATRIX:
    print("✅ Done. (Embeddings only)")
else:
    # ---------- 3) Build full similarity matrix (GPU-tiled) ----------
    print("📐 Building full similarity matrix (tiled GPU matmul)…")
    # Write to memmap so we don't need all RAM at once
    if OUT_SIM.exists():
        os.remove(OUT_SIM)
    sim_mm = np.memmap(OUT_SIM, mode="w+", dtype=EMB_DTYPE, shape=(K, K))

    # Move embeddings to GPU
    emb_t = torch.from_numpy(emb_fp16).to(device)
    # even though stored as fp16, matmul benefits from amp autocast
    n_blocks = math.ceil(K / BLOCK)
    print(f"➡ Using BLOCK={BLOCK} → {n_blocks}×{n_blocks} tiles")

    for bi in tqdm(range(0, K, BLOCK), desc="Row blocks"):
        i_end = min(bi + BLOCK, K)
        Ei = emb_t[bi:i_end]  # (Bi, d)
        for bj in range(0, K, BLOCK):
            j_end = min(bj + BLOCK, K)
            Ej = emb_t[bj:j_end]  # (Bj, d)
            with torch.inference_mode():
                if use_amp:
                    with torch.autocast(device_type="cuda", dtype=amp_dtype):
                        S = Ei @ Ej.T  # cosine = dot (embeddings are normalized)
                else:
                    S = Ei @ Ej.T
            # clamp tiny negatives due to numeric noise; cast to host
            S = torch.clamp(S, min=0).to(torch.float16 if EMB_DTYPE==np.float16 else torch.float32).cpu().numpy()
            sim_mm[bi:i_end, bj:j_end] = S
            del S
        gc.collect()
        if device == "cuda":
            torch.cuda.empty_cache()

    # ensure diagonal ~1
    np.fill_diagonal(sim_mm, np.float16(1.0) if EMB_DTYPE==np.float16 else 1.0)
    sim_mm.flush()
    del sim_mm
    print(f"💾 Saved similarity matrix (memmap): {OUT_SIM}  (shape {K}×{K}, dtype={EMB_DTYPE.__name__})")

    # Optional CSV if small K
    if K <= CSV_IF_K_LE:
        print("📝 Writing CSV matrix (small K)…")
        sim_mm = np.memmap(OUT_SIM, mode="r", dtype=EMB_DTYPE, shape=(K, K))
        sim_df = pd.DataFrame(sim_mm.astype(np.float32), index=keywords, columns=keywords)
        sim_df.to_csv(OUT_CSV, index=True)
        del sim_mm, sim_df
        print(f"💾 Also saved CSV: {OUT_CSV}")
    else:
        print(f"ℹ️ Matrix too large for CSV (K={K} > {CSV_IF_K_LE}). Use the .npy memmap and keyword_index.csv.")

    # cleanup
    del emb_fp16, emb_t
    gc.collect()
    if device == "cuda":
        torch.cuda.empty_cache()

    print("✅ Done. (Embeddings + full matrix)")


📥 Loading cleaned data…
🔎 Keeping keywords with ≥110 distinct weeks…


/tmp/ipython-input-1824595605.py:58: DeprecationWarning: The argument `streaming=True` is deprecated and is being replaced by the `engine` argument.
  .collect(streaming=True)
/tmp/ipython-input-1824595605.py:58: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  .collect(streaming=True)


✅ Kept 1,811 keywords.
💾 Wrote index mapping: /content/drive/MyDrive/colab_data/cleaned_cpu/keyword_index.csv
🧠 Loading model 'sentence-transformers/all-MiniLM-L6-v2' on cpu…
🔧 Computing embeddings…


Encoding keywords:   0%|          | 0/2 [00:00<?, ?it/s]

💾 Saved embeddings: /content/drive/MyDrive/colab_data/cleaned_cpu/keyword_embeddings.npy  (shape (1811, 384), dtype=float16)
📐 Building full similarity matrix (tiled GPU matmul)…
➡ Using BLOCK=4096 → 1×1 tiles


Row blocks:   0%|          | 0/1 [00:00<?, ?it/s]

💾 Saved similarity matrix (memmap): /content/drive/MyDrive/colab_data/cleaned_cpu/keyword_similarity_matrix.npy  (shape 1811×1811, dtype=float16)
📝 Writing CSV matrix (small K)…
💾 Also saved CSV: /content/drive/MyDrive/colab_data/cleaned_cpu/keyword_similarity_matrix.csv
✅ Done. (Embeddings + full matrix)


In [ ]:
# === Weighted semantic metrics (top 25% by SIMILARITY of UNIQUE neighbors) ===
# Inputs:
#   /content/drive/MyDrive/colab_data/weekly_processed.parquet  (week, keyword)
#   /content/drive/MyDrive/colab_data/keyword_index.csv         (index, keyword, n_weeks)
#   /content/drive/MyDrive/colab_data/keyword_similarity_matrix.npy  (KxK, float16)
# Output:
#   /content/drive/MyDrive/colab_data/weekly_semantic_metrics_weighted.parquet

# !pip -q install polars pyarrow tqdm

from pathlib import Path
import math
import numpy as np
import polars as pl
import pandas as pd
from tqdm.auto import tqdm
from datetime import date, timedelta

DATA_DIR   = Path("/content/drive/MyDrive/colab_data/cleaned_cpu")
IN_PARQUET = DATA_DIR / "weekly_processed.parquet"
IDX_CSV    = DATA_DIR / "keyword_index.csv"
SIM_NPY    = DATA_DIR / "keyword_similarity_matrix.npy"
OUT_PARQ   = DATA_DIR / "weekly_semantic_metrics_weighted.parquet"

TOP_FRAC = 0.25  # select top 25% most similar UNIQUE neighbors

def prev_week_iso(ww_yyyy: str) -> str | None:
    try:
        w, y = ww_yyyy.split("-")
        d = date.fromisocalendar(int(y), int(w), 1)
        d_prev = d - timedelta(days=7)
        iso = d_prev.isocalendar()
        return f"{iso.week:02d}-{iso.year}"
    except Exception:
        return None

def topk_by_similarity_weighted_mean(sim_vals: np.ndarray, counts: np.ndarray, frac: float) -> tuple[float, int]:
    """
    Select top-k *by similarity* among UNIQUE neighbors, where k = ceil(frac * M) and M=len(sim_vals).
    Then compute a weighted average of those k using 'counts' as weights.
    Return (weighted_mean, total_count_used). If no candidates, returns (nan, 0).
    """
    M = sim_vals.size
    if M == 0:
        return float("nan"), 0
    k = int(math.ceil(frac * M))
    k = max(1, min(k, M))  # ensure at least 1 and at most M

    order = np.argsort(-sim_vals)  # descending similarity
    sel = order[:k]

    w = counts[sel].astype(np.int64, copy=False)
    s = sim_vals[sel].astype(np.float32, copy=False)
    total = int(w.sum())
    if total <= 0:
        return float("nan"), 0
    return float((s * w).sum() / total), total

# --- load inputs ---
df = pl.read_parquet(IN_PARQUET).select("week", "keyword").with_columns(
    pl.col("week").cast(pl.Utf8), pl.col("keyword").cast(pl.Utf8)
)
idx_df = pd.read_csv(IDX_CSV)
keywords = idx_df["keyword"].tolist()
kw2idx = {kw: i for i, kw in enumerate(keywords)}
K = len(keywords)
sim = np.memmap(SIM_NPY, mode="r", dtype=np.float16, shape=(K, K))

# Keep only rows whose keyword is in our matrix (the 1060 keywords)
df = df.filter(pl.col("keyword").is_in(keywords))

# Per-week multiplicities (how often each keyword appears that week)
wk_counts = (
    df.group_by(["week", "keyword"])
      .len()
      .rename({"len": "count"})
)

# Weeks in ascending order
weeks = wk_counts.select("week").unique().sort("week").to_series().to_list()

# Build week -> arrays (indices, counts, keywords) and idx->pos map
week_idx, week_cnt, week_kws, week_posmap = {}, {}, {}, {}
for w in weeks:
    sub = wk_counts.filter(pl.col("week") == w)
    kws = sub["keyword"].to_list()
    cnt = np.asarray(sub["count"].to_numpy(), dtype=np.int32)  # counts of UNIQUE keywords in that week
    idxs = np.asarray([kw2idx[k] for k in kws], dtype=np.int32)
    week_kws[w] = kws
    week_cnt[w] = cnt
    week_idx[w] = idxs
    week_posmap[w] = {idxs[i]: i for i in range(len(idxs))}

# --- compute metrics ---
records = []
for w in tqdm(weeks, desc="Weeks"):
    idxs_this = week_idx[w]
    cnts_this = week_cnt[w]
    kws_this  = week_kws[w]
    n = len(idxs_this)

    # THIS WEEK (exclude self by -inf on diagonal)
    if n >= 1:
        S_this = np.asarray(sim[np.ix_(idxs_this, idxs_this)], dtype=np.float32)
        if n > 0:
            np.fill_diagonal(S_this, -np.inf)
    else:
        S_this = np.empty((0, 0), dtype=np.float32)

    # LAST WEEK slice
    w_prev = prev_week_iso(w)
    if w_prev in week_idx:
        idxs_prev = week_idx[w_prev]
        cnts_prev = week_cnt[w_prev]
        pos_prev  = week_posmap[w_prev]
        S_last = np.asarray(sim[np.ix_(idxs_this, idxs_prev)], dtype=np.float32)
    else:
        idxs_prev = np.empty((0,), dtype=np.int32)
        cnts_prev = np.empty((0,), dtype=np.int32)
        pos_prev  = {}
        S_last = np.empty((n, 0), dtype=np.float32)

    for i in range(n):
        kw_i = kws_this[i]

        # ---- THIS WEEK: select top-25% most similar UNIQUE neighbors; weight by their counts ----
        if n >= 2:
            sim_row = S_this[i]                 # similarities to unique neighbors this week
            counts  = cnts_this.copy()
            counts[i] = 0                       # exclude self
            mask = counts > 0
            mean_this, used_this = topk_by_similarity_weighted_mean(sim_row[mask], counts[mask], TOP_FRAC)
            avg_sim_this = mean_this
            n_sim_this = used_this             # sum of counts of the selected neighbors
        else:
            avg_sim_this = float("nan")
            n_sim_this = 0

        # ---- LAST WEEK: top-25% most similar UNIQUE neighbors; weight by last week's counts ----
        if idxs_prev.size > 0:
            sim_row_prev = S_last[i]
            counts_prev  = cnts_prev.copy()
            # exclude same keyword if present last week
            idx_i = kw2idx[kw_i]
            jprev = pos_prev.get(idx_i, None)
            if jprev is not None:
                counts_prev[jprev] = 0
            mask_prev = counts_prev > 0
            mean_last, used_last = topk_by_similarity_weighted_mean(sim_row_prev[mask_prev], counts_prev[mask_prev], TOP_FRAC)
            avg_sim_last = mean_last
            n_sim_last = used_last
        else:
            avg_sim_last = float("nan")
            n_sim_last = 0

        records.append((
            w, kw_i,
            float(avg_sim_this), float(avg_sim_last),
            int(n_sim_this), int(n_sim_last)
        ))

# Assemble and write
out = pl.DataFrame(
    records,
    schema=[
        ("week", pl.Utf8),
        ("keyword", pl.Utf8),
        ("avg_sim_top25_this_week", pl.Float32),
        ("avg_sim_top25_last_week", pl.Float32),
        ("n_sim_this_week", pl.Int32),
        ("n_sim_last_week", pl.Int32),
    ]
)

# Join onto unique (week, keyword) pairs (no aggregation of your original rows)
base = wk_counts.select("week", "keyword")
res = base.join(out, on=["week", "keyword"], how="left")

res.write_parquet(OUT_PARQ, compression="zstd", statistics=True)
print(f"✅ Done. Wrote: {OUT_PARQ}")
print(res.head(10))


Weeks:   0%|          | 0/127 [00:00<?, ?it/s]

/tmp/ipython-input-764023035.py:163: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  out = pl.DataFrame(


✅ Done. Wrote: /content/drive/MyDrive/colab_data/cleaned_cpu/weekly_semantic_metrics_weighted.parquet
shape: (10, 6)
┌─────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬────────────────┐
│ week    ┆ keyword         ┆ avg_sim_top25_t ┆ avg_sim_top25_l ┆ n_sim_this_week ┆ n_sim_last_wee │
│ ---     ┆ ---             ┆ his_week        ┆ ast_week        ┆ ---             ┆ k              │
│ str     ┆ str             ┆ ---             ┆ ---             ┆ i32             ┆ ---            │
│         ┆                 ┆ f32             ┆ f32             ┆                 ┆ i32            │
╞═════════╪═════════════════╪═════════════════╪═════════════════╪═════════════════╪════════════════╡
│ 07-2021 ┆ car rental ct   ┆ 0.679071        ┆ 0.676402        ┆ 23964           ┆ 22449          │
│ 36-2021 ┆ sixt rent       ┆ 0.489679        ┆ 0.488562        ┆ 62096           ┆ 60780          │
│         ┆ chicago         ┆                 ┆                 ┆          

In [ ]:
# === Attach weighted semantic metrics to every original row (no aggregation) ===
# Reads:
#   /content/drive/MyDrive/colab_data/weekly_processed.parquet
#   /content/drive/MyDrive/colab_data/weekly_semantic_metrics_weighted.parquet
# Writes:
#   /content/drive/MyDrive/colab_data/weekly_rows_enriched.parquet

# !pip -q install polars pyarrow

from pathlib import Path
import polars as pl

DATA_DIR = Path("/content/drive/MyDrive/colab_data/cleaned_cpu")
IN_ROWS  = DATA_DIR / "weekly_processed.parquet"
IN_METR  = DATA_DIR / "weekly_semantic_metrics_weighted.parquet"
OUT_FILE = DATA_DIR / "weekly_rows_enriched.parquet"

if not IN_ROWS.exists():
    raise FileNotFoundError(IN_ROWS)
if not IN_METR.exists():
    raise FileNotFoundError(IN_METR)

# lazy scans (fast & memory-friendly)
lf_rows = pl.scan_parquet(str(IN_ROWS))
lf_metr = pl.scan_parquet(str(IN_METR)).select(
    pl.col("week").cast(pl.Utf8),
    pl.col("keyword").cast(pl.Utf8),
    "avg_sim_top25_this_week",
    "avg_sim_top25_last_week",
    "n_sim_this_week",
    "n_sim_last_week",
)

# ensure join keys are Utf8 on both sides
lf_rows = lf_rows.with_columns(
    pl.col("week").cast(pl.Utf8, strict=False),
    pl.col("keyword").cast(pl.Utf8, strict=False),
)

# left-join metrics onto every original row (no aggregation)
enriched_rows = lf_rows.join(lf_metr, on=["week", "keyword"], how="left")

# write out
enriched_rows.sink_parquet(
    str(OUT_FILE),
    compression="zstd",
    statistics=True,
    row_group_size=8192,
)

print(f"✅ Wrote: {OUT_FILE}")
print(pl.read_parquet(OUT_FILE, n_rows=10))


✅ Wrote: /content/drive/MyDrive/colab_data/cleaned_cpu/weekly_rows_enriched.parquet
shape: (10, 11)
┌─────────┬────────────┬────────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ week    ┆ keyword    ┆ devicecate ┆ adcost   ┆ … ┆ avg_sim_t ┆ avg_sim_t ┆ n_sim_thi ┆ n_sim_las │
│ ---     ┆ ---        ┆ gory       ┆ ---      ┆   ┆ op25_this ┆ op25_last ┆ s_week    ┆ t_week    │
│ str     ┆ str        ┆ ---        ┆ f32      ┆   ┆ _week     ┆ _week     ┆ ---       ┆ ---       │
│         ┆            ┆ str        ┆          ┆   ┆ ---       ┆ ---       ┆ i32       ┆ i32       │
│         ┆            ┆            ┆          ┆   ┆ f32       ┆ f32       ┆           ┆           │
╞═════════╪════════════╪════════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 01-2021 ┆ rent       ┆ desktop    ┆ 1.91     ┆ … ┆ 0.667828  ┆ 0.665255  ┆ 35888     ┆ 14713     │
│ 01-2021 ┆ discount   ┆ desktop    ┆ 2.35     ┆ … ┆ null      ┆ null      ┆ null      ┆ nul

In [ ]:
# !pip -q install polars pyarrow

from pathlib import Path
import polars as pl

BASE = Path("/content/drive/MyDrive/colab_data")
IN_ENRICHED = BASE / "cleaned_cpu/weekly_rows_enriched.parquet"   # your current file
IDX_CSV     = BASE / "keyword_index.csv"                          # created during embeddings step
OUT_FILTER  = BASE / "cleaned_cpu/weekly_rows_enriched_filtered.parquet"

# Load the keyword whitelist (≥120 weeks)
kw_idx = pl.read_csv(IDX_CSV, infer_schema_length=0).select(pl.col("keyword").cast(pl.Utf8))
kw_set = kw_idx["keyword"].to_list()
print(f"✅ Keywords in metrics: {len(kw_set):,}")

# Lazy scan enriched rows and filter to keywords in the whitelist
lf = pl.scan_parquet(str(IN_ENRICHED)).with_columns(
    pl.col("keyword").cast(pl.Utf8, strict=False),
    pl.col("week").cast(pl.Utf8, strict=False),
)

# Optional: show before counts
before_rows = lf.select(pl.len()).collect().item()
before_unique_kw = lf.select(pl.col("keyword").n_unique()).collect().item()
print(f"📊 Before: {before_rows:,} rows, {before_unique_kw:,} unique keywords")

lf_filt = lf.filter(pl.col("keyword").is_in(kw_set))

# Optional: ensure metrics columns exist (if not, no-op)
metric_cols = ["avg_sim_top25_this_week","avg_sim_top25_last_week","n_sim_this_week","n_sim_last_week"]
for c in metric_cols:
    if c not in lf_filt.collect_schema().names():
        lf_filt = lf_filt.with_columns(pl.lit(None).alias(c))

# Write filtered parquet
lf_filt.sink_parquet(
    str(OUT_FILTER),
    compression="zstd",
    statistics=True,
    row_group_size=8192,
)

# Quick sanity report
df_head = pl.read_parquet(OUT_FILTER, n_rows=5)
after_rows = pl.read_parquet(OUT_FILTER, columns=["keyword"]).height
after_unique_kw = pl.read_parquet(OUT_FILTER, columns=["keyword"]).select(pl.col("keyword").n_unique()).item()

print(f"✅ Wrote: {OUT_FILTER}")
print(f"📊 After:  {after_rows:,} rows, {after_unique_kw:,} unique keywords")
print(df_head)


✅ Keywords in metrics: 1,066
📊 Before: 18,326,056 rows, 68,534 unique keywords
✅ Wrote: /content/drive/MyDrive/colab_data/cleaned_cpu/weekly_rows_enriched_filtered.parquet
📊 After:  3,117,852 rows, 108 unique keywords
shape: (5, 11)
┌─────────┬────────────┬────────────┬────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ week    ┆ keyword    ┆ devicecate ┆ adcost ┆ … ┆ avg_sim_to ┆ avg_sim_to ┆ n_sim_thi ┆ n_sim_las │
│ ---     ┆ ---        ┆ gory       ┆ ---    ┆   ┆ p25_this_w ┆ p25_last_w ┆ s_week    ┆ t_week    │
│ str     ┆ str        ┆ ---        ┆ f32    ┆   ┆ eek        ┆ eek        ┆ ---       ┆ ---       │
│         ┆            ┆ str        ┆        ┆   ┆ ---        ┆ ---        ┆ i32       ┆ i32       │
│         ┆            ┆            ┆        ┆   ┆ f32        ┆ f32        ┆           ┆           │
╞═════════╪════════════╪════════════╪════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 02-2021 ┆ car rental ┆ desktop    ┆ 1.46   ┆ … ┆ 0.666556 

In [ ]:
import polars as pl
from pathlib import Path

FILE = Path("/content/drive/MyDrive/colab_data/cleaned_cpu/weekly_rows_enriched.parquet")

# lazy scan for metadata
lf = pl.scan_parquet(str(FILE))
df = lf.collect()  # load to memory for inspection

# shape
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns\n")

# number of nulls per column
nulls = df.null_count().to_pandas().T
print("🔍 Null counts per column:\n")
print(nulls)

# unique value counts
unique_counts = {
    c: df[c].n_unique() for c in df.columns
}
print("\n🧩 Unique value counts:\n")
for k, v in unique_counts.items():
    print(f"{k:30s}: {v}")

# optional: sample rows
print("\n📋 Sample rows:")
print(df.head(5))


Shape: 18,326,056 rows × 11 columns

🔍 Null counts per column:

                               0
week                           0
keyword                        0
devicecategory                 0
adcost                         0
adclicks                       0
impressions                    0
searchtype                     0
avg_sim_top25_this_week  7874729
avg_sim_top25_last_week  7874729
n_sim_this_week          7874729
n_sim_last_week          7874729

🧩 Unique value counts:

week                          : 127
keyword                       : 68534
devicecategory                : 3
adcost                        : 923877
adclicks                      : 3599
impressions                   : 6281
searchtype                    : 2
avg_sim_top25_this_week       : 213683
avg_sim_top25_last_week       : 212619
n_sim_this_week               : 34639
n_sim_last_week               : 34663

📋 Sample rows:
shape: (5, 11)
┌─────────┬────────────┬────────────┬────────┬───┬────────────┬────────────

In [ ]:
### Google collab

# === Build CSV of keywords (>=120 distinct weeks) with global similarity score ===
# Works in Google Colab (local runtime). Expects Parquet files under /content/data
# Output: /content/data/keywords_120w_similarity.csv

# !pip -q install polars pyarrow scikit-learn tqdm

import os
from pathlib import Path
import numpy as np
import polars as pl
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm

DATA_DIR = Path("/content/data")
OUT_CSV = DATA_DIR / "keywords_120w_similarity.csv"

if not DATA_DIR.exists():
    raise FileNotFoundError(f"Data folder not found: {DATA_DIR}. Place your parquet files there.")

print("🔎 Scanning parquet files (only 'keyword' and 'week')...")
# Use Polars lazy scan for speed; skip macOS junk (.__ / __MACOSX) via glob pattern
lazy = pl.scan_parquet(str(DATA_DIR / "**" / "*.parquet"), allow_missing_columns=True)

# Ensure columns exist and are strings
cols = lazy.columns
necessary = []
for c in ("keyword", "week"):
    if c in cols:
        necessary.append(c)
    else:
        raise ValueError(f"Required column '{c}' not found in parquet schema. Found: {cols}")

lazy_needed = lazy.select(
    pl.col("keyword").cast(pl.Utf8).alias("keyword"),
    pl.col("week").cast(pl.Utf8).alias("week"),
)

print("🧮 Counting distinct weeks per keyword...")
kw_weeks = (
    lazy_needed
    .unique(subset=["keyword", "week"], maintain_order=False)  # dedup pairs
    .group_by("keyword")
    .agg(pl.n_unique("week").alias("n_weeks"))
)

print("🚿 Filtering to keywords with ≥ 120 distinct weeks...")
kw_valid = kw_weeks.filter(pl.col("n_weeks") >= 120).collect(streaming=True)

if kw_valid.height == 0:
    raise RuntimeError("No keywords appear in ≥ 120 distinct weeks. Check your data or threshold.")

keywords = kw_valid["keyword"].to_list()
n_weeks_vec = kw_valid["n_weeks"].to_list()
K = len(keywords)
print(f"✅ Kept {K} keywords meeting the ≥120-week criterion.")

# --- Build TF-IDF on the kept keywords and compute global similarity score ---
# Similarity score definition: average cosine similarity to ALL other kept keywords.
# (TF-IDF vectors are L2-normalized, so dot-products are cosine similarities.)
print("🧠 Computing TF-IDF (char 3–5 grams) and global similarity scores...")

# Use tqdm just for user feedback (small K finishes quickly)
tqdm.write("Vectorizing keywords...")
vect = TfidfVectorizer(analyzer="char", ngram_range=(3, 5), lowercase=True, min_df=1, norm="l2")
X = vect.fit_transform(keywords)  # shape (K, D), sparse CSR

# For typical K after filtering (hundreds), dense similarity is fine & fastest.
# If K is huge, switch to a streaming/approx path—but usually unnecessary here.
tqdm.write("Computing pairwise cosine similarities...")
sim_dense = (X @ X.T).toarray()  # (K, K), diagonal = 1.0
np.fill_diagonal(sim_dense, 0.0)

similarity_score = sim_dense.sum(axis=1) / np.maximum(1, (K - 1))  # avoid div by 0

# Build result table
result = pl.DataFrame({
    "keyword": keywords,
    "n_weeks": n_weeks_vec,
    "similarity_score": similarity_score.astype(float),
})

# Sort by highest similarity (optional, nice for inspection)
result = result.sort("similarity_score", descending=True)

# Save CSV
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
result.write_csv(OUT_CSV)
print(f"💾 Saved: {OUT_CSV}")

# Preview first rows
print(result.head(10))


🔎 Scanning parquet files (only 'keyword' and 'week')...
🧮 Counting distinct weeks per keyword...
🚿 Filtering to keywords with ≥ 120 distinct weeks...


/tmp/ipython-input-1193218733.py:27: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  cols = lazy.columns
/tmp/ipython-input-1193218733.py:49: DeprecationWarning: The argument `streaming=True` is deprecated and is being replaced by the `engine` argument.
  kw_valid = kw_weeks.filter(pl.col("n_weeks") >= 120).collect(streaming=True)
/tmp/ipython-input-1193218733.py:49: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  kw_

✅ Kept 241 keywords meeting the ≥120-week criterion.
🧠 Computing TF-IDF (char 3–5 grams) and global similarity scores...
Vectorizing keywords...
Computing pairwise cosine similarities...
💾 Saved: /content/data/keywords_120w_similarity.csv
shape: (10, 3)
┌─────────────────────────────────┬─────────┬──────────────────┐
│ keyword                         ┆ n_weeks ┆ similarity_score │
│ ---                             ┆ ---     ┆ ---              │
│ str                             ┆ i64     ┆ f64              │
╞═════════════════════════════════╪═════════╪══════════════════╡
│ car rental airport orlando      ┆ 127     ┆ 0.133853         │
│ car rental airport hertz        ┆ 125     ┆ 0.12762          │
│ car rental airport thrifty      ┆ 127     ┆ 0.124072         │
│ car rental airport seattle      ┆ 127     ┆ 0.117246         │
│ car rental airport hertz orlan… ┆ 127     ┆ 0.112357         │
│ car rental airport houston      ┆ 127     ┆ 0.112045         │
│ car rental airport at orlando

## The data transformations to be made

In [ ]:


df['date'] = pd.to_datetime(df['date'], errors='coerce', infer_datetime_format=True)
if df['date'].isna().all():
    raise ValueError("Could not parse any dates in 'date' column.")

iso = df['date'].dt.isocalendar()  # returns DataFrame with 'year','week','day'
df['week'] = iso['week'].astype(int).astype(str).str.zfill(2) + '-' + iso['year'].astype(str)

df.head(10)


C:\Users\sfrey\AppData\Local\Temp\ipykernel_23212\778183764.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'], errors='coerce', infer_datetime_format=True)


,date,keyword,admatchedquery,addestinationurl,devicecategory,adcost,adclicks,impressions,exchangerateusd,searchtype,cost_per_click,domain,week
0,2021-01-07,+car +rent,car rentals medford oregon,https://www.hertz.com/,desktop,1.91,1.0,1.0,1.0,Branded Search,1.910,hertz,01-2021
1,2021-01-13,+avis +car +rental,avis car rental rocky mount nc,https://www.avis.com/en/home,desktop,1.59,1.0,2.0,1.0,Generic Search,1.590,avis,02-2021
2,2021-01-15,budget car rentals,cheap car rentals,https://www.budget.ca/en/home,desktop,15.31,2.0,25.0,1.0,Generic Search,7.655,budget,02-2021
3,2021-01-14,+rental +car,car rental san jose del cabo airport,https://www.hertz.com/,desktop,0.98,1.0,1.0,1.0,Branded Search,0.980,hertz,02-2021
4,2021-01-07,discount auto rental,denver car rental,https://www.thrifty.com/,desktop,2.35,1.0,3.0,1.0,Branded Search,2.350,thrifty,01-2021
5,2021-01-07,hertz rent a car,hertz rental cars,https://www.hertz.com/,tablet,1.18,2.0,2.0,1.0,Branded Search,0.590,hertz,01-2021
6,2021-01-13,+car +rentals +denver +airport,denver colorado airport car rental,https://www.thrifty.com/Deals/Featured/101621....,mobile,3.95,1.0,1.0,1.0,Generic Search,3.950,thrifty,02-2021
7,2021-01-04,+car +rentals,herts rent a car,https://www.hertz.com/,mobile,1.82,1.0,1.0,1.0,Branded Search,1.820,hertz,01-2021
8,2021-01-11,+avis +car +rentals,avis mexico car rental,https://www.avis.ca/en/home,mobile,0.49,1.0,1.0,1.0,Generic Search,0.490,avis,02-2021
9,2021-01-11,budget car rental columbus,budget car rental columbus ohio,https://www.budget.com/en/offers/us-offers/boo...,mobile,2.26,2.0,7.0,1.0,Generic Search,1.130,budget,02-2021


In [ ]:
# --- installs (first run) ---
# %pip install -q "dask[complete]" pyarrow fastparquet

from pathlib import Path
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.distributed import Client
import pyarrow.dataset as ds

PARQUET_FOLDER = Path(r"C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english")
if not PARQUET_FOLDER.is_dir():
    raise NotADirectoryError(f"Not a directory: {PARQUET_FOLDER}")

# Start local Dask cluster (progress + dashboard)
client = Client()
print(client)

# -----------------------------
# 1) Safely discover schema (skip junk files)
# -----------------------------
IGNORE_PREFIXES = ["__MACOSX", "._", "_", "."]

dataset = ds.dataset(
    str(PARQUET_FOLDER),
    format="parquet",
    exclude_invalid_files=True,   # <- skip non-parquet junk
    ignore_prefixes=IGNORE_PREFIXES,  # <- skip hidden/underscore dirs/files
)

schema = dataset.schema
all_cols = schema.names

DROP_COLS = {'admatchedquery', 'addestinationurl', 'exchangerateusd', 'date'}
read_cols = [c for c in all_cols if c not in DROP_COLS] + (['date'] if 'date' in all_cols else [])
read_cols = list(dict.fromkeys(read_cols))

# -----------------------------
# 2) Read with the same ignore rules
# -----------------------------
ddf = dd.read_parquet(
    str(PARQUET_FOLDER),
    engine="pyarrow",
    columns=read_cols,
    # Make PyArrow’s dataset loader ignore the junk at read time, too:
    exclude_invalid_files=True,
    ignore_prefixes=IGNORE_PREFIXES,
    # If you have many tiny files, uncomment:
    # aggregate_files=True,
    # split_row_groups=True,
)

# -----------------------------
# 3) Transform
# -----------------------------
def _transform_partition(pdf: pd.DataFrame) -> pd.DataFrame:
    if 'date' not in pdf.columns:
        raise ValueError("Partition missing 'date' column")

    pdf['date'] = pd.to_datetime(pdf['date'], errors='coerce')
    if pdf['date'].isna().all():
        raise ValueError("Could not parse any dates in 'date'")

    iso = pdf['date'].dt.isocalendar()
    week_str = iso['week'].astype('UInt32').astype(str).str.zfill(2)
    year_str = iso['year'].astype('UInt32').astype(str)
    pdf['week'] = (week_str + '-' + year_str).astype('string')

    to_drop = [c for c in ('admatchedquery', 'addestinationurl', 'exchangerateusd', 'date') if c in pdf.columns]
    if to_drop:
        pdf = pdf.drop(columns=to_drop)
    return pdf

base_meta = ddf._meta.copy()
base_meta['week'] = pd.Series([], dtype="string")
for c in ('admatchedquery', 'addestinationurl', 'exchangerateusd', 'date'):
    if c in base_meta.columns:
        base_meta = base_meta.drop(columns=[c])

ddf_transformed = ddf.map_partitions(_transform_partition, meta=base_meta)

# -----------------------------
# 4) Progress + write
# -----------------------------
with ProgressBar():
    ddf_transformed = ddf_transformed.persist()

print("npartitions:", ddf_transformed.npartitions)
print("columns:", list(ddf_transformed.columns))
display(ddf_transformed.head(20))

OUT = PARQUET_FOLDER.parent / "multilanguage_english_weekly_processed"
with ProgressBar():
    ddf_transformed.to_parquet(
        str(OUT),
        engine="pyarrow",
        write_index=False,
        overwrite=True,
        compression="zstd",
        coerce_timestamps="ms",
    )
print("Wrote processed parquet to:", OUT)


c:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\.venv\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61038 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:61041' processes=4 threads=16, memory=14.80 GiB>
npartitions: 2205
columns: ['keyword', 'devicecategory', 'adcost', 'adclicks', 'impressions', 'searchtype', 'cost_per_click', 'domain', 'week']


,keyword,devicecategory,adcost,adclicks,impressions,searchtype,cost_per_click,domain,week
0,renting,desktop,3.430000,1.0,1.0,Branded Search,3.430000,hertz,01-2021
1,hire,mobile,0.171203,1.0,2.0,Branded Search,0.171203,economybookings,01-2021
2,rent thrifty,mobile,0.030000,3.0,2.0,Branded Search,0.010000,thrifty,01-2021
3,car rental budget new orleans,mobile,0.750000,1.0,1.0,Generic Search,0.750000,budget,02-2021
4,car rental four wheel drive,desktop,2.370000,1.0,1.0,Branded Search,2.370000,hertz,02-2021
5,car rental thrifty locations,mobile,0.270000,1.0,1.0,Branded Search,0.270000,thrifty,01-2021
6,car rental thrifty jfk,tablet,0.170000,1.0,1.0,Branded Search,0.170000,thrifty,02-2021
7,ace rentals perth,mobile,1.009853,1.0,1.0,Generic Search,1.009853,economybookings,02-2021
8,car rental thrifty atlanta,desktop,4.560000,1.0,4.0,Branded Search,4.560000,thrifty,04-2021
9,cheap cars orlando,desktop,4.290000,1.0,1.0,Generic Search,4.290000,dollar,06-2021


Wrote processed parquet to: C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english_weekly_processed


2025-11-06 05:10:36,202 - distributed.scheduler - WARNING - Worker failed to heartbeat for 3660s; attempting restart: <WorkerState 'tcp://127.0.0.1:61061', name: 0, status: running, memory: 550, processing: 0>
2025-11-06 05:10:36,209 - distributed.scheduler - WARNING - Worker failed to heartbeat for 3660s; attempting restart: <WorkerState 'tcp://127.0.0.1:61064', name: 2, status: running, memory: 531, processing: 0>
2025-11-06 05:10:36,214 - distributed.scheduler - WARNING - Worker failed to heartbeat for 3660s; attempting restart: <WorkerState 'tcp://127.0.0.1:61065', name: 3, status: running, memory: 537, processing: 0>
2025-11-06 05:10:36,332 - distributed.scheduler - WARNING - Worker failed to heartbeat for 3659s; attempting restart: <WorkerState 'tcp://127.0.0.1:61070', name: 1, status: running, memory: 587, processing: 0>
2025-11-06 05:10:36,482 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:61070' caused the cluster to lose already computed task(s), which w

In [ ]:
# === Keyword similarity via TF-IDF char n-grams (No torch, No onnx) ===
# %pip install -q "dask[complete]" scikit-learn pyarrow fastparquet tqdm

from pathlib import Path
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# ---------------- Config ----------------
PARQUET_IN = Path(r"C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english_weekly_processed")
OUT = PARQUET_IN.parent / "multilanguage_english_weekly_enhanced"

TOPK = 10                 # neighbors to average
MAX_EXACT_K = 2000        # switch to NN for within-week if unique keywords > this
MAX_EXACT_K_CROSS = 3000  # switch to NN for cross-week if last-week unique keywords > this

# Dask: 1 worker avoids cross-process pickling issues for sklearn objects
cluster = LocalCluster(n_workers=1, threads_per_worker=2, processes=False)
client = Client(cluster)
print(client)

# ---------------- Helpers ----------------
CLICK_CANDIDATES = ["clicks", "total_clicks", "click_count", "click", "Clicks", "TotalClicks"]

def get_last_week(week_str: str) -> str | None:
    """Convert 'WW-YYYY' (ISO-like) -> previous week 'WW-YYYY' (assumes 52 weeks if crossing year)."""
    try:
        s = str(week_str)
        w, y = s.split("-")
        w, y = int(w), int(y)
        return f"{w-1:02d}-{y}" if w > 1 else f"52-{y-1}"
    except Exception:
        return None

def pick_weight_col(df: pd.DataFrame) -> str | None:
    for c in CLICK_CANDIDATES:
        if c in df.columns:
            return c
    return None

def tfidf_fit_transform(labels_a: list[str], labels_b: list[str] | None = None):
    """Returns (X_a, X_b, vectorizer). X_b is None if labels_b is None."""
    vect = TfidfVectorizer(analyzer="char", ngram_range=(3, 5), lowercase=True, min_df=1, norm="l2")
    if labels_b is None:
        X_a = vect.fit_transform(labels_a)
        return X_a, None, vect
    else:
        X_all = vect.fit_transform(labels_a + labels_b)
        N = len(labels_a)
        X_a = X_all[:N]
        X_b = X_all[N:]
        return X_a, X_b, vect

def weighted_topk_avg_from_indices(sim_vals: np.ndarray, idxs: np.ndarray, labels: list[str], weight_map: dict[str, float], k: int) -> float:
    """Compute weighted average of top-k similarities given neighbor indices."""
    if sim_vals.size == 0:
        return np.nan
    # ensure at most k
    sim_vals = sim_vals[:k]
    idxs = idxs[:k]
    ws = np.array([float(weight_map.get(labels[i], 0.0)) for i in idxs], dtype="float64")
    if ws.sum() > 0:
        return float((sim_vals * ws).sum() / ws.sum())
    return float(sim_vals.mean()) if sim_vals.size else np.nan

def compute_weekly_metrics(pdf: pd.DataFrame) -> pd.DataFrame:
    """
    For each week in this partition:
      - comp_this_week: count occurrences of each keyword in week
      - avg_sim_this_week: mean cosine sim of a keyword to other unique keywords in the same week (exact for small weeks; NaN for huge weeks to avoid O(K^2))
      - avg_sim_last_week: mean cosine sim to unique keywords from the previous week (exact for small; NaN for huge)
      - avg_top10_sim_this_week_wt: weighted avg over TOP-10 most similar in same week
      - avg_top10_sim_last_week_wt: weighted avg over TOP-10 most similar in previous week
        (weights = total clicks per keyword if available, else frequency in that week)
    """
    pdf = pdf.copy()
    if "keyword" not in pdf.columns or "week" not in pdf.columns:
        return pdf

    pdf["keyword"] = pdf["keyword"].astype(str)
    pdf["week"] = pdf["week"].astype(str)

    weight_col = pick_weight_col(pdf)

    out = pdf.copy()
    out["comp_this_week"] = np.int64(0)
    out["avg_sim_this_week"] = np.nan
    out["avg_sim_last_week"] = np.nan
    out["avg_top10_sim_this_week_wt"] = np.nan
    out["avg_top10_sim_last_week_wt"] = np.nan

    weeks = pd.unique(pdf["week"])
    for week in weeks:
        if pd.isna(week):
            continue

        # Slice for this week
        mask_w = (pdf["week"] == week)
        df_w = pdf.loc[mask_w, ["keyword"] + ([weight_col] if weight_col else [])]

        # Counts and weights (within week)
        counts = df_w["keyword"].value_counts()
        out.loc[mask_w, "comp_this_week"] = out.loc[mask_w, "keyword"].map(counts).astype("int64")

        if weight_col:
            weights_this = df_w.groupby("keyword", as_index=True)[weight_col].sum().to_dict()
        else:
            weights_this = counts.to_dict()

        # Unique keywords this week
        uniq = pd.unique(df_w["keyword"]).tolist()
        K = len(uniq)
        if K == 0:
            continue

        # ---- Within-week similarity ----
        if K == 1:
            # only itself; leave NaNs for sim columns and continue
            pass
        else:
            # Fit TF-IDF once
            X, _, _ = tfidf_fit_transform(uniq, None)

            # TOP-10 weighted within-week via NearestNeighbors (always efficient)
            # Use n_neighbors = min(K, TOPK+1) because first neighbor is self
            k_nn = min(K, TOPK + 1)
            nn_w = NearestNeighbors(metric="cosine", n_neighbors=k_nn).fit(X)
            dist_w, idx_w = nn_w.kneighbors(X, return_distance=True)       # dist in [0,2], cosine sim = 1 - dist
            # remove self (first neighbor)
            dist_w = dist_w[:, 1:]
            idx_w  = idx_w[:, 1:]
            sim_w = 1.0 - dist_w
            top10_wt_within = {
                kw: weighted_topk_avg_from_indices(sim_w[i], idx_w[i], uniq, weights_this, TOPK)
                for i, kw in enumerate(uniq)
            }
            out.loc[mask_w, "avg_top10_sim_this_week_wt"] = out.loc[mask_w, "keyword"].map(top10_wt_within)

            # Exact full-matrix mean only for small K
            if K <= MAX_EXACT_K:
                # (X @ X.T).A gives cosine sims because TF-IDF is L2-normalized
                sim_full = (X @ X.T).toarray()
                np.fill_diagonal(sim_full, 0.0)
                mean_sims = sim_full.sum(axis=1) / (K - 1)
                mean_dict = dict(zip(uniq, mean_sims))
                out.loc[mask_w, "avg_sim_this_week"] = out.loc[mask_w, "keyword"].map(mean_dict)
            # else: keep NaN to avoid O(K^2)

        # ---- Cross-week similarity (vs previous week) ----
        last_week = get_last_week(week)
        if last_week and (pdf["week"] == last_week).any():
            df_lw = pdf.loc[pdf["week"] == last_week, ["keyword"] + ([weight_col] if weight_col else [])]
            uniq_lw = pd.unique(df_lw["keyword"]).tolist()
            L = len(uniq_lw)
            if L > 0:
                # Weights for last week
                if weight_col:
                    weights_last = df_lw.groupby("keyword", as_index=True)[weight_col].sum().to_dict()
                else:
                    weights_last = df_lw["keyword"].value_counts().to_dict()

                # Fit TF-IDF on union (align features)
                X_a, X_b, _ = tfidf_fit_transform(uniq, uniq_lw)

                # Adaptive path:
                if L <= MAX_EXACT_K_CROSS and K <= MAX_EXACT_K_CROSS:
                    # Exact mean over ALL last-week keywords
                    cross = (X_a @ X_b.T).toarray()  # (K, L)
                    out.loc[mask_w, "avg_sim_last_week"] = out.loc[mask_w, "keyword"].map(
                        dict(zip(uniq, cross.mean(axis=1)))
                    )
                    # Weighted TOP-10 using argsort on cross
                    # Get top indices per row (exclude if L < TOPK)
                    top_idx = np.argpartition(-cross, kth=min(TOPK-1, L-1), axis=1)[:, :min(TOPK, L)]
                    # refine order
                    row_ids = np.arange(cross.shape[0])[:, None]
                    ordered = np.argsort(-cross[row_ids, top_idx], axis=1)
                    top_idx = top_idx[row_ids, ordered]
                    top_vals = cross[row_ids, top_idx]
                    top10_last_wt = {
                        kw: weighted_topk_avg_from_indices(top_vals[i], top_idx[i], uniq_lw, weights_last, TOPK)
                        for i, kw in enumerate(uniq)
                    }
                    out.loc[mask_w, "avg_top10_sim_last_week_wt"] = out.loc[mask_w, "keyword"].map(top10_last_wt)
                else:
                    # Fast: use NN on last-week space, query current week
                    nn = NearestNeighbors(metric="cosine", n_neighbors=min(TOPK, max(1, L))).fit(X_b)
                    dist, idx = nn.kneighbors(X_a, return_distance=True)
                    sim = 1.0 - dist
                    top10_last_wt = {
                        kw: weighted_topk_avg_from_indices(sim[i], idx[i], uniq_lw, weights_last, TOPK)
                        for i, kw in enumerate(uniq)
                    }
                    out.loc[mask_w, "avg_top10_sim_last_week_wt"] = out.loc[mask_w, "keyword"].map(top10_last_wt)
                    # avg_sim_last_week (unweighted) is skipped for large weeks to avoid heavy compute

    return out

# --------- Execute ----------
if not PARQUET_IN.exists():
    raise FileNotFoundError(f"Input parquet not found: {PARQUET_IN}")

print("Reading input parquet…")
ddf = dd.read_parquet(str(PARQUET_IN))

# --- GLOBAL FILTER: keep only keywords that appear in >= 120 distinct weeks ---
print("Filtering keywords with fewer than 120 weeks...")
keyword_week_counts = (
    ddf[["keyword", "week"]]
    .drop_duplicates()
    .groupby("keyword")["week"]
    .nunique()
)
valid_keywords = keyword_week_counts[keyword_week_counts >= 120].compute()
valid_set = set(valid_keywords.index)
ddf = ddf[ddf["keyword"].isin(valid_set)]
print(f"✅ Kept {len(valid_set):,} keywords that appear in ≥120 weeks.")

# --- Dask meta with new columns ---
meta = ddf._meta.copy()
meta["comp_this_week"] = pd.Series(dtype="int64")
meta["avg_sim_this_week"] = pd.Series(dtype="float64")
meta["avg_sim_last_week"] = pd.Series(dtype="float64")
meta["avg_top10_sim_this_week_wt"] = pd.Series(dtype="float64")
meta["avg_top10_sim_last_week_wt"] = pd.Series(dtype="float64")

print("Computing metrics with TF-IDF (char n-grams)…")
ddf_enhanced = ddf.map_partitions(compute_weekly_metrics, meta=meta)

# Persist & write with visible progress
with ProgressBar():
    ddf_enhanced = ddf_enhanced.persist()

OUT.mkdir(parents=True, exist_ok=True)
print(f"Writing to {OUT} …")
with ProgressBar():
    ddf_enhanced.to_parquet(
        str(OUT),
        engine="pyarrow",
        write_index=False,
        overwrite=True
    )

print("✅ Done. Saved enhanced parquet.")
print("\nSample:")
print(ddf_enhanced.head(10))

client.close()
cluster.close()


c:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\.venv\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 51312 instead
  warnings.warn(


<Client: 'inproc://192.168.1.78/21628/9' processes=1 threads=2, memory=14.80 GiB>
Reading input parquet…
Filtering keywords with fewer than 120 weeks...


2025-11-06 03:36:25,022 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle fcc141bc2a8b413eb881dd7ebbe3c469 initialized by task ('shuffle-transfer-fcc141bc2a8b413eb881dd7ebbe3c469', 241) executed on worker inproc://192.168.1.78/21628/12
2025-11-06 03:40:41,362 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 852aa206282f6d6abd3fd4bb1e190650 initialized by task ('shuffle-transfer-852aa206282f6d6abd3fd4bb1e190650', 99) executed on worker inproc://192.168.1.78/21628/12
2025-11-06 03:43:44,363 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle fcc141bc2a8b413eb881dd7ebbe3c469 deactivated due to stimulus 'task-finished-1762397024.3595936'
2025-11-06 03:44:22,984 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 852aa206282f6d6abd3fd4bb1e190650 deactivated due to stimulus 'task-finished-1762397062.9825048'


✅ Kept 255 keywords that appear in ≥120 weeks.
Computing metrics with TF-IDF (char n-grams)…
Writing to C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english_weekly_enhanced …


2025-11-06 04:09:34,266 - distributed.scheduler - WARNING - Worker failed to heartbeat for 1279s; removing: <WorkerState 'inproc://192.168.1.78/21628/12', name: 0, status: running, memory: 205, processing: 3>
2025-11-06 04:09:34,347 - distributed.scheduler - WARNING - Workers ['inproc://192.168.1.78/21628/12'] do not use a nanny and will be terminated without restarting them
2025-11-06 04:09:34,443 - distributed.scheduler - WARNING - Removing worker 'inproc://192.168.1.78/21628/12' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('toparquetdata-b10e76ec25b3ba073301106d370c4e61', 962), ('read_parquet-fused-compute_weekly_metrics-3ca6ca44081792d0b9542c2120152976', 954), ('read_parquet-fused-compute_weekly_metrics-3ca6ca44081792d0b9542c2120152976', 963), ('read_parquet-fused-compute_weekly_metrics-3ca6ca44081792d0b9542c2120152976', 908), ('read_parquet-fused-compute_weekly_metrics-3ca6ca44081792d0b9542c2120152976', 972), ('read_parquet-fused-compu

KeyboardInterrupt: 

In [ ]:
# %pip install -q "dask[complete]" scikit-learn pyarrow fastparquet tqdm

from pathlib import Path
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm

# ---------------- Config ----------------
PARQUET_IN = Path(r"C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english_weekly_processed")
OUT = PARQUET_IN.parent / "multilanguage_english_weekly_enhanced"
TOPK = 10

# Use Dask's threaded (no cluster) scheduler to avoid worker heartbeat quirks here
dask.config.set(scheduler="threads")

CLICK_CANDIDATES = ["adclicks", "clicks", "total_clicks", "click_count", "Clicks", "TotalClicks"]
WEIGHT_FALLBACK = "impressions"  # fallback if clicks not present

def get_last_week(week_str: str):
    """'WW-YYYY' -> previous week in same format (assumes 52 weeks when crossing year)."""
    try:
        s = str(week_str)
        w, y = s.split("-")
        w, y = int(w), int(y)
        return f"{w-1:02d}-{y}" if w > 1 else f"52-{y-1}"
    except Exception:
        return None

def pick_weight_col(columns):
    for c in CLICK_CANDIDATES:
        if c in columns:
            return c
    return WEIGHT_FALLBACK if WEIGHT_FALLBACK in columns else None

def tfidf(labels_a, labels_b=None):
    """Return (X_a, X_b or None, vectorizer). Char 3-5 grams, L2-normalized so dot=cosine."""
    vect = TfidfVectorizer(analyzer="char", ngram_range=(3,5), lowercase=True, min_df=1, norm="l2")
    if labels_b is None:
        Xa = vect.fit_transform(labels_a)
        return Xa, None, vect
    Xall = vect.fit_transform(labels_a + labels_b)
    n = len(labels_a)
    return Xall[:n], Xall[n:], vect

def topk_weighted_avg_from_idx(sim_vals, idxs, labels, weight_map, k):
    sim_vals = sim_vals[:k]
    idxs = idxs[:k]
    ws = np.array([float(weight_map.get(labels[i], 0.0)) for i in idxs], dtype="float64")
    if ws.sum() > 0:
        return float((sim_vals * ws).sum() / ws.sum())
    return float(sim_vals.mean()) if sim_vals.size else np.nan

# --------- Execute ----------
if not PARQUET_IN.exists():
    raise FileNotFoundError(f"Input parquet not found: {PARQUET_IN}")

print("Reading input parquet…")
# Read only what we need; keep other columns for final merge/write
ddf_raw = dd.read_parquet(str(PARQUET_IN), engine="pyarrow")
needed = {"week", "keyword", "impressions"} | set(CLICK_CANDIDATES)
present_needed = [c for c in ddf_raw.columns if c in needed]
ddf = ddf_raw[present_needed].astype({"week": "string", "keyword": "string"})

# ---- Global filter: keep keywords with >=120 distinct weeks ----
print("Filtering keywords with fewer than 120 weeks...")
keyword_week_counts = (
    ddf[["keyword", "week"]]
    .drop_duplicates()
    .groupby("keyword")["week"]
    .nunique()
)
valid_keywords = keyword_week_counts[keyword_week_counts >= 120].compute()  # small vector
valid_set = set(valid_keywords.index)
ddf = ddf[ddf["keyword"].isin(valid_set)]
print(f"✅ Kept {len(valid_set):,} keywords that appear in ≥120 weeks.")

# ---- Pick weight column (clicks if available, else impressions, else frequency) ----
weight_col = pick_weight_col(ddf.columns)
if weight_col:
    print(f"Using weights from column: {weight_col}")
else:
    print("No clicks/impressions column found — falling back to frequency weights.")

# ---- Aggregate per (week, keyword): counts and weights ----
print("Aggregating per (week, keyword)…")
aggs = {"keyword": "size"}  # will rename to comp_this_week
if weight_col:
    aggs[weight_col] = "sum"
elif "impressions" in ddf.columns:
    aggs["impressions"] = "sum"

wk_kw_agg = (
    ddf.groupby(["week", "keyword"])
       .agg(aggs)
       .rename(columns={"keyword": "comp_this_week"})
       .reset_index()
)

with ProgressBar():
    wk_kw_pdf = wk_kw_agg.compute()  # pandas DataFrame, typically small now

# Prepare weight maps per week
if weight_col and weight_col in wk_kw_pdf.columns:
    wk_kw_pdf["weight_week"] = wk_kw_pdf[weight_col].astype("float64")
elif "impressions" in wk_kw_pdf.columns:
    wk_kw_pdf["weight_week"] = wk_kw_pdf["impressions"].astype("float64")
else:
    wk_kw_pdf["weight_week"] = wk_kw_pdf["comp_this_week"].astype("float64")

# ---- Build per-week lists for driver-side similarity ----
weeks = sorted(wk_kw_pdf["week"].unique().tolist())
print(f"Weeks to process: {len(weeks)}")

# Result containers
rows = []

# Driver-side loop with progress
for w in tqdm(weeks, desc="Computing per-week similarities"):
    this_df = wk_kw_pdf[wk_kw_pdf["week"] == w]
    uniq = this_df["keyword"].astype(str).tolist()
    if len(uniq) == 0:
        continue

    # weights for this week (per keyword)
    w_this = dict(zip(this_df["keyword"], this_df["weight_week"]))
    # frequency for this week (per keyword)
    f_this = dict(zip(this_df["keyword"], this_df["comp_this_week"]))

    # Within-week features
    Xa, _, _ = tfidf(uniq, None)

    # Within-week nearest neighbors (cosine)
    k_nn = min(len(uniq), TOPK + 1)  # +1 to include self
    nn_w = NearestNeighbors(metric="cosine", n_neighbors=k_nn).fit(Xa)
    dist_w, idx_w = nn_w.kneighbors(Xa, return_distance=True)
    # remove self (first neighbor)
    dist_w = dist_w[:, 1:]
    idx_w = idx_w[:, 1:]
    sim_w = 1.0 - dist_w

    # Unweighted mean to all others (approximate via many neighbors if small; otherwise use all available)
    # For <= 2000 uniques this equals exact mean if we set neighbors=len(uniq)
    if k_nn == len(uniq):
        # exact mean over all others: average neighbors excluding self
        mean_sim_this = sim_w.mean(axis=1)
    else:
        # large weeks (unlikely after 255-keyword filter); use TOPK mean as proxy
        mean_sim_this = sim_w.mean(axis=1)

    top10_wt_this = [
        topk_weighted_avg_from_idx(sim_w[i], idx_w[i], uniq, w_this, TOPK)
        for i in range(len(uniq))
    ]

    # Cross-week vs last week
    lw = get_last_week(w)
    if lw and (lw in weeks):
        last_df = wk_kw_pdf[wk_kw_pdf["week"] == lw]
        uniq_lw = last_df["keyword"].astype(str).tolist()
        if len(uniq_lw) > 0:
            w_last = dict(zip(last_df["keyword"], last_df["weight_week"]))
            Xa2, Xb2, _ = tfidf(uniq, uniq_lw)
            # Use NN from current to last week
            k2 = min(len(uniq_lw), TOPK)
            nn_lw = NearestNeighbors(metric="cosine", n_neighbors=k2).fit(Xb2)
            dist_lw, idx_lw = nn_lw.kneighbors(Xa2, return_distance=True)
            sim_lw = 1.0 - dist_lw

            # Unweighted mean to all last-week keywords (efficient: use matmul if small; else use k2 mean)
            if len(uniq_lw) <= 4000:
                # exact mean via sparse matmul -> dense row means
                cross_full = (Xa2 @ Xb2.T).toarray()
                mean_sim_last = cross_full.mean(axis=1)
            else:
                # fallback to k-NN mean
                mean_sim_last = sim_lw.mean(axis=1)

            top10_wt_last = [
                topk_weighted_avg_from_idx(sim_lw[i], idx_lw[i], uniq_lw, w_last, TOPK)
                for i in range(len(uniq))
            ]
        else:
            mean_sim_last = [np.nan] * len(uniq)
            top10_wt_last = [np.nan] * len(uniq)
    else:
        mean_sim_last = [np.nan] * len(uniq)
        top10_wt_last = [np.nan] * len(uniq)

    # Append rows
    rows.extend(
        {
            "week": w,
            "keyword": kw,
            "comp_this_week": int(f_this.get(kw, 0)),
            "avg_sim_this_week": float(mean_sim_this[i]),
            "avg_sim_last_week": float(mean_sim_last[i]),
            "avg_top10_sim_this_week_wt": float(top10_wt_this[i]),
            "avg_top10_sim_last_week_wt": float(top10_wt_last[i]),
        }
        for i, kw in enumerate(uniq)
    )

# ---- Build result table and merge back to original filtered data ----
metrics_df = pd.DataFrame(rows)
# Merge metrics onto filtered original rows (so every row of the week/keyword gets the metrics)
dd_metrics = dd.from_pandas(metrics_df, npartitions=max(1, len(metrics_df) // 200_000))

# Reattach to full rowset (but only for kept keywords)
ddf_filtered = ddf_raw[ddf_raw["keyword"].isin(valid_set)]
# Make sure week/keyword are string
ddf_filtered = ddf_filtered.assign(
    week=ddf_filtered["week"].astype("string"),
    keyword=ddf_filtered["keyword"].astype("string"),
)

print("Merging metrics back to rows…")
with ProgressBar():
    ddf_final = ddf_filtered.merge(dd_metrics, on=["week", "keyword"], how="left").persist()

OUT.mkdir(parents=True, exist_ok=True)
print(f"Writing to {OUT} …")
with ProgressBar():
    ddf_final.to_parquet(
        str(OUT),
        engine="pyarrow",
        write_index=False,
        overwrite=True
    )

print("✅ Done. Saved enhanced parquet.")
print("\nSample:")
print(ddf_final.head(10))


c:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading input parquet…
Filtering keywords with fewer than 120 weeks...


RuntimeError: P2P 56ecd341aca03face5b8805672c1c80e failed during transfer phase

In [ ]:
# === Keyword similarity via TF-IDF char n-grams (No torch, No onnx) ===
%pip install -q "dask[complete]" scikit-learn pyarrow fastparquet tqdm

from pathlib import Path
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
from sklearn.feature_extraction.text import TfidfVectorizer

# ---------------- Config ----------------
PARQUET_IN = Path(r"C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english_weekly_processed")
OUT = PARQUET_IN.parent / "multilanguage_english_weekly_enhanced"

# Dask: keep it simple, 1 worker avoids cross-process issues
cluster = LocalCluster(n_workers=1, threads_per_worker=2, processes=False)
client = Client(cluster)

def get_last_week(week_str: str):
    """Convert 'WW-YYYY' -> previous week in same format."""
    try:
        s = str(week_str)
        w, y = s.split("-")
        w, y = int(w), int(y)
        return f"{w-1:02d}-{y}" if w > 1 else f"52-{y-1}"
    except Exception:
        return None

def tfidf_cosine(texts_a, texts_b=None):
    """
    Build a TF-IDF (char 3-5 grams) matrix and return cosine sim:
      - if texts_b is None -> returns (N,N) sim within texts_a
      - else -> returns (N,M) sim between texts_a and texts_b
    """
    # Character n-grams handle typos/variants; L2 norm means dot = cosine
    vect = TfidfVectorizer(analyzer="char", ngram_range=(3,5), lowercase=True, min_df=1, norm="l2")
    if texts_b is None:
        X = vect.fit_transform(texts_a)          # (N, D)
        sim = (X @ X.T).toarray()                # (N, N) cosine
        return sim
    else:
        X = vect.fit_transform(list(texts_a) + list(texts_b))  # fit on union to align features
        N = len(texts_a)
        X_a = X[:N]
        X_b = X[N:]
        sim = (X_a @ X_b.T).toarray()           # (N, M) cosine
        return sim

def compute_weekly_metrics(pdf: pd.DataFrame) -> pd.DataFrame:
    """
    For each week in this partition:
      - comp_this_week: count occurrences of each keyword in week
      - avg_sim_this_week: mean cosine sim of a keyword to other unique keywords in the same week
      - avg_sim_last_week: mean cosine sim of a keyword to unique keywords from the previous week
    """
    pdf = pdf.copy()
    if "keyword" not in pdf.columns or "week" not in pdf.columns:
        return pdf

    pdf["keyword"] = pdf["keyword"].astype(str)
    pdf["week"] = pdf["week"].astype(str)

    result = pdf.copy()
    result["comp_this_week"] = np.int64(0)
    result["avg_sim_this_week"] = np.float64(0.0)
    result["avg_sim_last_week"] = np.nan

    weeks = pd.unique(pdf["week"])
    for week in weeks:
        if pd.isna(week):
            continue
        week_mask = (pdf["week"] == week)
        week_df = pdf.loc[week_mask, ["keyword"]]

        # 1) competition count
        counts = week_df["keyword"].value_counts()
        result.loc[week_mask, "comp_this_week"] = result.loc[week_mask, "keyword"].map(counts).astype("int64")

        # 2) within-week similarity on unique keywords
        unique_kw = pd.unique(week_df["keyword"]).tolist()
        if len(unique_kw) > 1:
            sim = tfidf_cosine(unique_kw)  # (K,K)
            K = len(unique_kw)
            # mean over other keywords (exclude diagonal)
            mean_sims = (sim.sum(axis=1) - np.diag(sim)) / (K - 1)
            sim_dict = dict(zip(unique_kw, mean_sims))
            result.loc[week_mask, "avg_sim_this_week"] = result.loc[week_mask, "keyword"].map(sim_dict)

        # 3) cross-week similarity vs last week
        last_week = get_last_week(week)
        if last_week and (pdf["week"] == last_week).any():
            lw_kw = pd.unique(pdf.loc[pdf["week"] == last_week, "keyword"]).tolist()
            if len(unique_kw) > 0 and len(lw_kw) > 0:
                cross = tfidf_cosine(unique_kw, lw_kw)  # (K, L)
                mean_cross = cross.mean(axis=1)
                cross_dict = dict(zip(unique_kw, mean_cross))
                result.loc[week_mask, "avg_sim_last_week"] = result.loc[week_mask, "keyword"].map(cross_dict)

    return result

# --------- Execute ----------
if not PARQUET_IN.exists():
    raise FileNotFoundError(f"Input parquet not found: {PARQUET_IN}")

print("Reading input parquet…")
ddf = dd.read_parquet(str(PARQUET_IN))

# meta with new cols
meta = ddf._meta.copy()
meta["comp_this_week"] = pd.Series(dtype="int64")
meta["avg_sim_this_week"] = pd.Series(dtype="float64")
meta["avg_sim_last_week"] = pd.Series(dtype="float64")

print("Computing metrics with TF-IDF (char n-grams)…")
ddf_enhanced = ddf.map_partitions(compute_weekly_metrics, meta=meta)

print(f"Writing to {OUT} …")
ddf_enhanced.to_parquet(
    str(OUT),
    engine="pyarrow",
    write_index=False,
    compute=True
)
print("✅ Done. Saved enhanced parquet.")

print("\nSample:")
print(ddf_enhanced.head(10))

client.close()
cluster.close()


Note: you may need to restart the kernel to use updated packages.


c:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\.venv\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57491 instead
  warnings.warn(


Reading input parquet…
Computing metrics with TF-IDF (char n-grams)…
Writing to C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english_weekly_enhanced …


Task exception was never retrieved
future: <Task finished name='Task-122124' coro=<Client._gather.<locals>.wait() done, defined at c:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\.venv\Lib\site-packages\distributed\client.py:2384> exception=AllExit()>
Traceback (most recent call last):
  File "c:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\.venv\Lib\site-packages\distributed\client.py", line 2393, in wait
    raise AllExit()
distributed.client.AllExit


KeyboardInterrupt: 

2025-11-06 04:09:34,248 - distributed.scheduler - WARNING - Worker failed to heartbeat for 1279s; removing: <WorkerState 'inproc://192.168.1.78/21628/4', name: 0, status: running, memory: 0, processing: 0>
2025-11-06 04:09:34,315 - distributed.scheduler - WARNING - Workers ['inproc://192.168.1.78/21628/4'] do not use a nanny and will be terminated without restarting them
2025-11-06 04:09:34,414 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'inproc://192.168.1.78/21628/4'.
2025-11-06 04:09:34,459 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import dask.dataframe as dd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

PARQUET_IN = Path(r"C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english_weekly_processed")
OUT = PARQUET_IN.parent / "multilanguage_english_weekly_enhanced"

from datetime import date, timedelta

def last_week_str(wwyyyy: str) -> str:
    """Convert 'WW-YYYY' to previous week 'WW-YYYY' using ISO weeks."""
    w_str, y_str = wwyyyy.split("-")
    y, w = int(y_str), int(w_str)
    # pick Sunday (7) to mimic W-SUN notion; 1 would be Monday
    d = date.fromisocalendar(y, w, 7)
    prev = d - timedelta(days=7)
    iy, iw, _ = prev.isocalendar()
    return f"{iw:02d}-{iy}"


# ---- Load minimal columns
ddf = dd.read_parquet(str(PARQUET_IN), columns=["week","keyword"], engine="pyarrow").astype({"week":"string","keyword":"string"})

# ---- Counts per (week, keyword) on CPU
wk_kw_counts = ddf.groupby(["week","keyword"]).size().compute().rename("comp_this_week")  # pandas Series
wk_uniques_df = wk_kw_counts.reset_index()[["week","keyword"]]
week_to_kws = wk_uniques_df.groupby("week")["keyword"].apply(list).to_dict()
week_list = sorted(week_to_kws.keys())

# ---- One global TF-IDF (float32), then slice per week
all_unique_keywords = wk_uniques_df["keyword"].unique().tolist()
vec = TfidfVectorizer(analyzer="char", ngram_range=(3,5), lowercase=True, min_df=1, norm="l2", dtype=np.float32)
vec.fit(all_unique_keywords)
X_all = vec.transform(all_unique_keywords).astype(np.float32)  # scipy CSR
kw2row = {kw:i for i, kw in enumerate(all_unique_keywords)}

def counts_array_for_week(w, kws):
    return np.fromiter((wk_kw_counts.get((w,k), 0) for k in kws), dtype=np.float32)

rows = []
prev = {"week": None, "X": None, "cnt": None}

for i, w in enumerate(tqdm(week_list, desc="Per-week metrics (fast)", ncols=90)):
    print(f"→ Processing week {i+1}/{len(week_list)}: {w}")
    kws = week_to_kws[w]
    idx = [kw2row[k] for k in kws]
    Xw = X_all[idx, :]
    cnt_w = counts_array_for_week(w, kws)

    K = Xw.shape[0]
    if K == 1:
        avg_sim_this = np.array([0.0], dtype=np.float32)
    else:
        t = Xw.T @ cnt_w                 # (D,)
        row_weighted = (Xw @ t).ravel()
        num = row_weighted - cnt_w
        den = cnt_w.sum() - cnt_w
        with np.errstate(divide="ignore", invalid="ignore"):
            avg_sim_this = np.where(den > 0, num / den, 0.0).astype(np.float32)

    lw = last_week_str(w)
    if prev["week"] == lw:
        Xp, cnt_p = prev["X"], prev["cnt"]
    else:
        if lw in week_to_kws:
            kws_p = week_to_kws[lw]
            idx_p = [kw2row[k] for k in kws_p]
            Xp = X_all[idx_p, :]
            cnt_p = counts_array_for_week(lw, kws_p)
        else:
            Xp = None; cnt_p = None

    if Xp is not None and cnt_p is not None and Xp.shape[0] > 0 and cnt_p.sum() > 0:
        v = Xp.T @ cnt_p
        avg_sim_last = (Xw @ v).ravel() / float(cnt_p.sum())
        avg_sim_last = avg_sim_last.astype(np.float32)
    else:
        avg_sim_last = np.full(K, np.nan, dtype=np.float32)

    rows.extend((w, kw, int(comp), float(st), float(sl) if np.isfinite(sl) else np.nan)
                for kw, comp, st, sl in zip(kws, cnt_w, avg_sim_this, avg_sim_last))

    prev = {"week": w, "X": Xw, "cnt": cnt_w}

metrics_df = pd.DataFrame(rows, columns=["week","keyword","comp_this_week","avg_sim_this_week","avg_sim_last_week"])

# ---- Broadcast merge (avoid 37M-row shuffle) + efficient write
ddf_full = dd.read_parquet(str(PARQUET_IN), engine="pyarrow").astype({"week":"string","keyword":"string"})
ddf_enhanced = ddf_full.map_partitions(lambda part: part.merge(metrics_df, on=["week","keyword"], how="left"))

ddf_enhanced = ddf_enhanced.categorize(columns=["week","keyword"])
OUT.parent.mkdir(parents=True, exist_ok=True)
ddf_enhanced.to_parquet(
    str(OUT),
    engine="pyarrow",
    write_index=False,
    compression="zstd",
    compression_level=3,
    row_group_size=1_000_000
)
print("✅ Done.")


KeyboardInterrupt: 

In [ ]:
ddf_enhanced.head(10)

,keyword,devicecategory,adcost,adclicks,impressions,searchtype,cost_per_click,domain,week,comp_this_week,avg_sim_this_week,avg_sim_last_week
0,renting,desktop,3.430000,1.0,1.0,Branded Search,3.430000,hertz,01-2021,1300,0.012110,0.054171
1,hire,mobile,0.171203,1.0,2.0,Branded Search,0.171203,economybookings,01-2021,993,0.015241,0.067009
2,rent thrifty,mobile,0.030000,3.0,2.0,Branded Search,0.010000,thrifty,01-2021,168,0.078560,0.099602
3,car rental budget new orleans,mobile,0.750000,1.0,1.0,Generic Search,0.750000,budget,02-2021,61,0.063484,0.076478
4,car rental four wheel drive,desktop,2.370000,1.0,1.0,Branded Search,2.370000,hertz,02-2021,21,0.028197,0.027115
5,car rental thrifty locations,mobile,0.270000,1.0,1.0,Branded Search,0.270000,thrifty,01-2021,16,0.097026,0.097054
6,car rental thrifty jfk,tablet,0.170000,1.0,1.0,Branded Search,0.170000,thrifty,02-2021,24,0.092170,0.092588
7,ace rentals perth,mobile,1.009853,1.0,1.0,Generic Search,1.009853,economybookings,02-2021,1,0.027978,0.031128
8,car rental thrifty atlanta,desktop,4.560000,1.0,4.0,Branded Search,4.560000,thrifty,04-2021,24,0.082485,0.080313
9,cheap cars orlando,desktop,4.290000,1.0,1.0,Generic Search,4.290000,dollar,06-2021,165,0.070288,0.056212


In [ ]:
import duckdb

IN_DIR = r"C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english_weekly_enhanced"
OUT    = r"C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\multilanguage_english_weekly_agg_new.parquet"

threads = os.cpu_count()  # use all logical cores

con = duckdb.connect()
con.execute(f"PRAGMA threads={threads};")
con.execute("PRAGMA memory_limit='10GB';")  # adjust to your RAM
con.execute("PRAGMA enable_object_cache=true;")

sql = f"""
CREATE OR REPLACE TABLE kw_agg AS
WITH base AS (
  SELECT
    CAST(week AS VARCHAR)    AS week,
    CAST(keyword AS VARCHAR) AS keyword,
    SUM(impressions)         AS impressions,
    SUM(adclicks)            AS adclicks,
    SUM(adcost)              AS adcost,
    ANY_VALUE(comp_this_week)     AS comp_this_week,
    ANY_VALUE(avg_sim_this_week)  AS avg_sim_this_week,
    ANY_VALUE(avg_sim_last_week)  AS avg_sim_last_week
  FROM parquet_scan('{IN_DIR.replace("\\","/")}/**/*.parquet')
  GROUP BY 1,2
),
counts AS (
  SELECT
    CAST(week AS VARCHAR)    AS week,
    CAST(keyword AS VARCHAR) AS keyword,
    -- absolute counts by device
    SUM(CASE WHEN lower(devicecategory)='desktop' THEN 1 ELSE 0 END) AS n_dev_desktop,
    SUM(CASE WHEN lower(devicecategory)='mobile'  THEN 1 ELSE 0 END) AS n_dev_mobile,
    SUM(CASE WHEN lower(devicecategory)='tablet'  THEN 1 ELSE 0 END) AS n_dev_tablet,
    -- absolute counts by searchtype (normalize spaces->underscores if needed)
    SUM(CASE WHEN lower(replace(searchtype,' ','_'))='branded_search' THEN 1 ELSE 0 END)  AS n_st_branded_search,
    SUM(CASE WHEN lower(replace(searchtype,' ','_'))='generic_search' THEN 1 ELSE 0 END)  AS n_st_generic_search
  FROM parquet_scan('{IN_DIR.replace("\\","/")}/**/*.parquet')
  GROUP BY 1,2
)
SELECT
  b.week,
  b.keyword,
  b.impressions,
  b.comp_this_week,
  CAST(b.avg_sim_this_week AS FLOAT) AS avg_sim_this_week,
  CAST(b.avg_sim_last_week AS FLOAT) AS avg_sim_last_week,
  b.adclicks,
  CAST(b.adcost AS FLOAT) AS adcost,
  CASE WHEN b.adclicks > 0
       THEN CAST(b.adcost AS DOUBLE)/b.adclicks
       ELSE NULL
  END AS cpc_week,
  CAST(n_dev_desktop AS INTEGER)      AS n_dev_desktop,
  CAST(n_dev_mobile  AS INTEGER)      AS n_dev_mobile,
  CAST(n_dev_tablet  AS INTEGER)      AS n_dev_tablet,
  CAST(n_st_branded_search AS INTEGER) AS n_st_branded_search,
  CAST(n_st_generic_search AS INTEGER) AS n_st_generic_search
FROM base b
JOIN counts c USING (week, keyword);

"""

con.execute(sql)

# Write single parquet
con.execute(f"""
COPY kw_agg TO '{OUT.replace("\\","/")}'
    (FORMAT PARQUET, COMPRESSION ZSTD);
""")
con.close()


In [ ]:
# drop adclicks and adcost from the loaded DataFrame and overwrite the parquet file
cols_to_drop = [c for c in ("adclicks", "adcost") if c in dftest.columns]
FINAL_FILE = r"C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\weekly_data_final_new.parquet"

if not cols_to_drop:
    print("No columns to drop:", ("adclicks", "adcost"))
else:
    df_new = dftest.drop(columns=cols_to_drop).copy()
    # overwrite the parquet file (uses pyarrow engine available in the notebook)
    df_new.to_parquet(str(FINAL_FILE), index=False, engine="pyarrow")
    print(f"Wrote {len(df_new)} rows to {FINAL_FILE} without columns: {cols_to_drop}")
    display(df_new.head())

Wrote 420366 rows to C:\Users\sfrey\Documents\privat\uni\thesis\master_thesis\data\weekly_data_final.parquet without columns: ['adclicks', 'adcost']


,week,keyword,impressions,comp_this_week,avg_sim_this_week,avg_sim_last_week,cpc_week,n_dev_desktop,n_dev_mobile,n_dev_tablet,n_st_branded_search,n_st_generic_search
0,01-2021,rent thrifty,310.0,168.0,0.078560,0.099602,0.644068,48,116,4,166,2
1,14-2021,car rental airport cheap at las vegas,2.0,1.0,0.057569,0.053771,1.850000,0,1,0,0,1
2,18-2021,car rental airport edinburgh,86.0,24.0,0.062028,0.060613,1.917924,16,8,0,22,2
3,31-2021,car rental thrifty tulsa,22.0,12.0,0.069974,0.069131,0.334286,6,6,0,12,0
4,29-2021,thrifty com,44.0,24.0,0.045092,0.039363,0.801000,8,16,0,24,0
